In [3]:
import pandas as pd
import re 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fnmatch
from yelpapi import YelpAPI
import json
from decimal import *
from pathlib import Path

%matplotlib inline

In [4]:
pov_data = pd.read_excel('../Data/poverty_rates.xlsx', sheet_name = 'Percents', header = 2)
pov_data

#bringing in data from census

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,0.026037,0.012483,0.045000,0.004918,0.034508,0.011601,0.034091,0.008723,...,0.036052,0.009398,0.024596,0.005037,0.036945,0.007138,0.012765,0.003430,0.028517,0.009377
1,.50 to .74,poverty,0.024342,0.012483,0.023456,0.003069,0.022094,0.010641,0.016845,0.005842,...,0.019741,0.007608,0.021070,0.004877,0.014934,0.004207,0.010619,0.003978,0.011689,0.004958
2,.75 to .99,poverty,0.022202,0.008382,0.037121,0.003978,0.026306,0.010788,0.023047,0.007602,...,0.034112,0.011686,0.020271,0.004767,0.024564,0.005882,0.008348,0.003414,0.015594,0.005010
3,1.00 to 1.24,low-income,0.030584,0.012305,0.031564,0.003722,0.026528,0.010641,0.041693,0.013684,...,0.021880,0.006415,0.024117,0.004423,0.023726,0.005643,0.021285,0.006578,0.020476,0.006423
4,1.25 to 1.49,low-income,0.033081,0.010700,0.039468,0.004527,0.027562,0.009163,0.030210,0.010083,...,0.039582,0.011885,0.034043,0.007765,0.029409,0.006859,0.010087,0.002945,0.023045,0.006860
5,1.50 to 1.74,low-income,0.038698,0.012037,0.038541,0.004502,0.034360,0.010345,0.048496,0.013404,...,0.032074,0.009597,0.026598,0.005283,0.027834,0.005523,0.012201,0.003759,0.025152,0.007039
6,1.75 to 1.84,low-income,0.019082,0.012127,0.016473,0.002711,0.020838,0.011158,0.018326,0.008083,...,0.013526,0.007260,0.014006,0.004374,0.010906,0.003370,0.008896,0.002631,0.009223,0.003468
7,1.85 to 1.99,low-income,0.029157,0.010878,0.020719,0.003376,0.031848,0.011158,0.025768,0.008283,...,0.021929,0.006663,0.024473,0.004755,0.020656,0.005762,0.012358,0.006061,0.013976,0.003879
8,2.00 to 2.99,NaN,0.169327,0.021400,0.146650,0.007731,0.202172,0.029262,0.197423,0.023648,...,0.179712,0.022576,0.178449,0.014190,0.147064,0.012102,0.086676,0.009163,0.156998,0.018035
9,3.00 to 3.99,NaN,0.192867,0.024253,0.134462,0.007917,0.196483,0.031848,0.165773,0.021407,...,0.167330,0.022327,0.161986,0.009325,0.172346,0.014156,0.099018,0.010431,0.156073,0.015004


In [5]:
pov_data.info()   #making sure numbers are the correct type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rate                   13 non-null     object 
 1   Income level           8 non-null      object 
 2   Cheatham County, TN    13 non-null     float64
 3   Error                  13 non-null     float64
 4   Davidson County, TN    13 non-null     float64
 5   Error.1                13 non-null     float64
 6   Dickson County, TN     13 non-null     float64
 7   Error.2                13 non-null     float64
 8   Maury County, TN       13 non-null     float64
 9   Error.3                13 non-null     float64
 10  Montgomery County, TN  13 non-null     float64
 11  Error.4                13 non-null     float64
 12  Robertson County, TN   13 non-null     float64
 13  Error.5                13 non-null     float64
 14  Rutherford County, TN  13 non-null     float64
 15  Error.6 

In [6]:
pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']] = pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']].apply(lambda x: x*100)
pov_data

#updating column titles

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.603656,1.248328,4.499965,0.491754,3.450824,1.160127,3.409091,0.872279,...,3.605172,0.939831,2.459580,0.503710,3.694547,0.713787,1.276489,0.343008,2.851711,0.937725
1,.50 to .74,poverty,2.434240,1.248328,2.345583,0.306947,2.209414,1.064066,1.684539,0.584187,...,1.974142,0.760816,2.106983,0.487739,1.493371,0.420696,1.061914,0.397826,1.168945,0.495838
2,.75 to .99,poverty,2.220241,0.838163,3.712135,0.397752,2.630607,1.078844,2.304738,0.760243,...,3.411238,1.168573,2.027127,0.476682,2.456385,0.588177,0.834808,0.341441,1.559449,0.500976
3,1.00 to 1.24,low-income,3.058404,1.230495,3.156434,0.372173,2.652775,1.064066,4.169334,1.368438,...,2.187966,0.641472,2.411666,0.442282,2.372645,0.564251,2.128526,0.657823,2.047580,0.642277
4,1.25 to 1.49,low-income,3.308070,1.069996,3.946822,0.452746,2.756226,0.916279,3.020967,1.008323,...,3.958230,1.188463,3.404344,0.776451,2.940883,0.685874,1.008661,0.294454,2.304491,0.685952
5,1.50 to 1.74,low-income,3.869817,1.203745,3.854098,0.450188,3.436045,1.034508,4.849552,1.340429,...,3.207360,0.959722,2.659836,0.528281,2.783372,0.552288,1.220104,0.375899,2.515158,0.703936
6,1.75 to 1.84,low-income,1.908159,1.212662,1.647280,0.271136,2.083795,1.115791,1.832586,0.808259,...,1.352561,0.726007,1.400560,0.437368,1.090619,0.336955,0.889627,0.263129,0.922310,0.346830
7,1.85 to 1.99,low-income,2.915738,1.087829,2.071889,0.337641,3.184808,1.115791,2.576825,0.828265,...,2.192939,0.666335,2.447295,0.475453,2.065597,0.576214,1.235767,0.606137,1.397595,0.387935
8,2.00 to 2.99,NaN,16.932679,2.139991,14.665013,0.773122,20.217247,2.926180,19.742318,2.364757,...,17.971159,2.257583,17.844857,1.418989,14.706410,1.210248,8.667596,0.916253,15.699825,1.803515
9,3.00 to 3.99,NaN,19.286670,2.425323,13.446179,0.791666,19.648267,3.184808,16.577305,2.140685,...,16.732969,2.232720,16.198585,0.932478,17.234573,1.415612,9.901796,1.043119,15.607337,1.500360


In [11]:
pov_data.round(2)  #rounding numbers to 2 decimal places

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.60,1.25,4.50,0.49,3.45,1.16,3.41,0.87,...,3.61,0.94,2.46,0.50,3.69,0.71,1.28,0.34,2.85,0.94
1,.50 to .74,poverty,2.43,1.25,2.35,0.31,2.21,1.06,1.68,0.58,...,1.97,0.76,2.11,0.49,1.49,0.42,1.06,0.40,1.17,0.50
2,.75 to .99,poverty,2.22,0.84,3.71,0.40,2.63,1.08,2.30,0.76,...,3.41,1.17,2.03,0.48,2.46,0.59,0.83,0.34,1.56,0.50
3,1.00 to 1.24,low-income,3.06,1.23,3.16,0.37,2.65,1.06,4.17,1.37,...,2.19,0.64,2.41,0.44,2.37,0.56,2.13,0.66,2.05,0.64
4,1.25 to 1.49,low-income,3.31,1.07,3.95,0.45,2.76,0.92,3.02,1.01,...,3.96,1.19,3.40,0.78,2.94,0.69,1.01,0.29,2.30,0.69
5,1.50 to 1.74,low-income,3.87,1.20,3.85,0.45,3.44,1.03,4.85,1.34,...,3.21,0.96,2.66,0.53,2.78,0.55,1.22,0.38,2.52,0.70
6,1.75 to 1.84,low-income,1.91,1.21,1.65,0.27,2.08,1.12,1.83,0.81,...,1.35,0.73,1.40,0.44,1.09,0.34,0.89,0.26,0.92,0.35
7,1.85 to 1.99,low-income,2.92,1.09,2.07,0.34,3.18,1.12,2.58,0.83,...,2.19,0.67,2.45,0.48,2.07,0.58,1.24,0.61,1.40,0.39
8,2.00 to 2.99,NaN,16.93,2.14,14.67,0.77,20.22,2.93,19.74,2.36,...,17.97,2.26,17.84,1.42,14.71,1.21,8.67,0.92,15.70,1.80
9,3.00 to 3.99,NaN,19.29,2.43,13.45,0.79,19.65,3.18,16.58,2.14,...,16.73,2.23,16.20,0.93,17.23,1.42,9.90,1.04,15.61,1.50


In [12]:
pov_data.drop(columns = ['Error', 'Error.1', 'Error.2', 'Error.3', 'Error.4', 'Error.5', 'Error.6', 'Error.7', 'Error.8', 'Error.9',])

,Rate,Income level,"Cheatham County, TN","Davidson County, TN","Dickson County, TN","Maury County, TN","Montgomery County, TN","Robertson County, TN","Rutherford County, TN","Sumner County, TN","Williamson County, TN","Wilson County, TN"
0,Under .50,poverty,2.603656,4.499965,3.450824,3.409091,4.119695,3.605172,2.459580,3.694547,1.276489,2.851711
1,.50 to .74,poverty,2.434240,2.345583,2.209414,1.684539,2.224522,1.974142,2.106983,1.493371,1.061914,1.168945
2,.75 to .99,poverty,2.220241,3.712135,2.630607,2.304738,3.210690,3.411238,2.027127,2.456385,0.834808,1.559449
3,1.00 to 1.24,low-income,3.058404,3.156434,2.652775,4.169334,3.355604,2.187966,2.411666,2.372645,2.128526,2.047580
4,1.25 to 1.49,low-income,3.308070,3.946822,2.756226,3.020967,3.841159,3.958230,3.404344,2.940883,1.008661,2.304491
5,1.50 to 1.74,low-income,3.869817,3.854098,3.436045,4.849552,4.669239,3.207360,2.659836,2.783372,1.220104,2.515158
6,1.75 to 1.84,low-income,1.908159,1.647280,2.083795,1.832586,1.776607,1.352561,1.400560,1.090619,0.889627,0.922310
7,1.85 to 1.99,low-income,2.915738,2.071889,3.184808,2.576825,3.139174,2.192939,2.447295,2.065597,1.235767,1.397595
8,2.00 to 2.99,NaN,16.932679,14.665013,20.217247,19.742318,20.261598,17.971159,17.844857,14.706410,8.667596,15.699825
9,3.00 to 3.99,NaN,19.286670,13.446179,19.648267,16.577305,15.562247,16.732969,16.198585,17.234573,9.901796,15.607337


In [10]:
pov_data.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/pov.csv')

In [6]:
with open('keys.json') as fi:
    credentials = json.load(fi)
    
#calling in yelp api and secret id

In [7]:
api_key = credentials['yelp_key']
client_id = credentials['yelp_id']
yelp_api = YelpAPI(api_key)

In [8]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
parks = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'park', location = location, search_limit = 20, offset = x)
        parks.append(response)
        print(len(parks))
        
#creating loop. made list of locations to run through. group is how many time to run through for the offset (only gives 20
# without offset changing). created list of parks to store found results in. 
#loop goes through to find parks in all the locations listed and is going through yelp until reaching the 500 limit
#then appending it to add it each loop to the parks list.
#printing list to check that it is changing after each loop

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [9]:
parks[1]     #checking to see what the parks list looks like. It is a list of dicts of 20 parks

{'businesses': [{'id': 'G3FSsyHWzJ-s4ndaC-C95Q',
   'alias': 'henslee-park-dickson',
   'name': 'Henslee Park',
   'image_url': '',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/henslee-park-dickson?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 1,
   'categories': [{'alias': 'parks', 'title': 'Parks'}],
   'rating': 5.0,
   'coordinates': {'latitude': 36.087676, 'longitude': -87.397686},
   'transactions': [],
   'location': {'address1': '800 Hwy 70 W',
    'address2': '',
    'address3': None,
    'city': 'Dickson',
    'zip_code': '37055',
    'country': 'US',
    'state': 'TN',
    'display_address': ['800 Hwy 70 W', 'Dickson, TN 37055']},
   'phone': '',
   'display_phone': '',
   'distance': 32357.461332730283},
  {'id': 'P7nobv_tPta4NCyC0mXrtw',
   'alias': 'oak-grove-war-memorial-walking-trail-park-oak-grove',
   'name': 'Oak Grove War Memorial Walking Trail Pa

In [10]:
parks_df = pd.DataFrame()

for park in range(len(parks)):
    df = pd.DataFrame(parks[park]['businesses'])
    parks_df = parks_df.append(df)
    print(parks_df.shape)
    
#creating a df for response to go into
#looking for parks in the range of the length of parks
#putting each dict within park into a df
#then appending parks_df so each dict that was turned into a df goes into the new parks_df each loop
#printing shape to see that it is grabbing more dicts from parks

(20, 15)
(40, 15)
(60, 15)
(80, 15)
(100, 15)
(120, 16)
(140, 16)
(160, 16)
(180, 16)
(200, 16)
(220, 16)
(240, 16)
(260, 16)
(280, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(288, 16)
(308, 16)
(328, 16)
(348, 16)
(368, 16)
(388, 16)
(408, 16)
(428, 16)
(448, 16)
(468, 16)
(488, 16)
(508, 16)
(528, 16)
(548, 16)
(568, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(572, 16)
(592, 16)
(612, 16)
(632, 16)
(652, 16)
(672, 16)
(692, 16)
(712, 16)
(732, 16)
(752, 16)
(772, 16)
(792, 16)
(812, 16)
(832, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(839, 16)
(859, 16)
(879, 16)
(899, 16)
(919, 16)
(939, 16)
(959, 16)
(979, 16)
(999, 16)
(1019, 16)
(1039, 16)
(1059, 16)
(1079, 16)
(1099, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16)
(1100, 16

In [11]:
parks_df   #checking to ensure it is a df and running correctly/how many rows

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,U_Ae6-egO2I0jd2jvGYdDw,el-paseo-nashville,El Paseo,https://s3-media2.fl.yelpcdn.com/bphoto/rHBH0l...,False,https://www.yelp.com/biz/el-paseo-nashville?ad...,28,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 36.1579438, 'longitude': -86.8490...","[delivery, pickup]","{'address1': '905 51st Ave N', 'address2': Non...",+16158669517,(615) 866-9517,48791.450177,NaN
8,7RV1xwWIT99llRoZ_caixA,stone-river-national-cemetery-murfreesboro,Stone River National Cemetery,https://s3-media1.fl.yelpcdn.com/bphoto/L7aXeX...,False,https://www.yelp.com/biz/stone-river-national-...,4,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 35.884486, 'longitude': -86.43472}",[],"{'address1': '3501 Old Nashville Hwy', 'addres...",+16158939501,(615) 893-9501,32226.373275,NaN
9,pDqFwkWgZ7L7vE5fGEEfRg,church-st-park-nashville,Church St. Park,,False,https://www.yelp.com/biz/church-st-park-nashvi...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.1626930236816, 'longitude': -8...",[],"{'address1': '600 Church St', 'address2': '', ...",,,42749.900544,NaN
10,SabmUBdLlLlwDTd1NB97BA,owen-peabody-park-nashville,Owen Peabody Park,https://s3-media3.fl.yelpcdn.com/bphoto/mL9bD7...,False,https://www.yelp.com/biz/owen-peabody-park-nas...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.15216, 'longitude': -86.79135}",[],"{'address1': 'Buddy Killen Cir', 'address2': '...",,,43633.736720,NaN


In [12]:
parks_df = parks_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,U_Ae6-egO2I0jd2jvGYdDw,el-paseo-nashville,El Paseo,https://s3-media2.fl.yelpcdn.com/bphoto/rHBH0l...,False,https://www.yelp.com/biz/el-paseo-nashville?ad...,28,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 36.1579438, 'longitude': -86.8490...","[delivery, pickup]","{'address1': '905 51st Ave N', 'address2': Non...",+16158669517,(615) 866-9517,48791.450177,NaN
2762,7RV1xwWIT99llRoZ_caixA,stone-river-national-cemetery-murfreesboro,Stone River National Cemetery,https://s3-media1.fl.yelpcdn.com/bphoto/L7aXeX...,False,https://www.yelp.com/biz/stone-river-national-...,4,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 35.884486, 'longitude': -86.43472}",[],"{'address1': '3501 Old Nashville Hwy', 'addres...",+16158939501,(615) 893-9501,32226.373275,NaN
2763,pDqFwkWgZ7L7vE5fGEEfRg,church-st-park-nashville,Church St. Park,,False,https://www.yelp.com/biz/church-st-park-nashvi...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.1626930236816, 'longitude': -8...",[],"{'address1': '600 Church St', 'address2': '', ...",,,42749.900544,NaN
2764,SabmUBdLlLlwDTd1NB97BA,owen-peabody-park-nashville,Owen Peabody Park,https://s3-media3.fl.yelpcdn.com/bphoto/mL9bD7...,False,https://www.yelp.com/biz/owen-peabody-park-nas...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.15216, 'longitude': -86.79135}",[],"{'address1': 'Buddy Killen Cir', 'address2': '...",,,43633.736720,NaN


In [13]:
parks_df[parks_df.duplicated(subset = 'name', keep = 'first')]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
88,PGY64SyinTS5jm-YgMb-9g,liberty-park-franklin,Liberty Park,https://s3-media2.fl.yelpcdn.com/bphoto/U1El6l...,False,https://www.yelp.com/biz/liberty-park-franklin...,5,"[{'alias': 'discgolf', 'title': 'Disc Golf'}]",3.5,"{'latitude': 35.9309170888114, 'longitude': -8...",[],"{'address1': '1669 McEwen Dr', 'address2': '',...",,,44090.968033,NaN
258,9kwQHZ28oi2wnVjpoV-eVw,williamson-county-parks-and-rec-dept-brentwood,Williamson County Parks & Rec Dept,https://s3-media3.fl.yelpcdn.com/bphoto/dvt489...,False,https://www.yelp.com/biz/williamson-county-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",3.5,"{'latitude': 36.0019912719727, 'longitude': -8...",[],"{'address1': '920 Heritage Way', 'address2': '...",+16153703471,(615) 370-3471,38874.553363,NaN
288,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,4675.104705,NaN
289,yu9e-_S7U97kleJQw0eOzQ,richland-creek-greenway-nashville,Richland Creek Greenway,https://s3-media4.fl.yelpcdn.com/bphoto/fiqTzB...,False,https://www.yelp.com/biz/richland-creek-greenw...,30,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 36.140481933298005, 'longitude': ...",[],"{'address1': '4601 Murphy Rd', 'address2': '',...",,,7350.416823,NaN
290,CRmZDx9rW3HFo0Z9FbfyEw,cumberland-park-nashville,Cumberland Park,https://s3-media1.fl.yelpcdn.com/bphoto/xRQAmw...,False,https://www.yelp.com/biz/cumberland-park-nashv...,32,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.1626862381255, 'longitude': -8...",[],"{'address1': '592 S 1st St', 'address2': '', '...",,,2839.804616,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,U_Ae6-egO2I0jd2jvGYdDw,el-paseo-nashville,El Paseo,https://s3-media2.fl.yelpcdn.com/bphoto/rHBH0l...,False,https://www.yelp.com/biz/el-paseo-nashville?ad...,28,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 36.1579438, 'longitude': -86.8490...","[delivery, pickup]","{'address1': '905 51st Ave N', 'address2': Non...",+16158669517,(615) 866-9517,48791.450177,NaN
2762,7RV1xwWIT99llRoZ_caixA,stone-river-national-cemetery-murfreesboro,Stone River National Cemetery,https://s3-media1.fl.yelpcdn.com/bphoto/L7aXeX...,False,https://www.yelp.com/biz/stone-river-national-...,4,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 35.884486, 'longitude': -86.43472}",[],"{'address1': '3501 Old Nashville Hwy', 'addres...",+16158939501,(615) 893-9501,32226.373275,NaN
2763,pDqFwkWgZ7L7vE5fGEEfRg,church-st-park-nashville,Church St. Park,,False,https://www.yelp.com/biz/church-st-park-nashvi...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.1626930236816, 'longitude': -8...",[],"{'address1': '600 Church St', 'address2': '', ...",,,42749.900544,NaN
2764,SabmUBdLlLlwDTd1NB97BA,owen-peabody-park-nashville,Owen Peabody Park,https://s3-media3.fl.yelpcdn.com/bphoto/mL9bD7...,False,https://www.yelp.com/biz/owen-peabody-park-nas...,2,"[{'alias': 'parks', 'title': 'Parks'}]",2.5,"{'latitude': 36.15216, 'longitude': -86.79135}",[],"{'address1': 'Buddy Killen Cir', 'address2': '...",,,43633.736720,NaN


In [14]:
parks_df[parks_df['name']== 'Shelby Park']   #checking random park to see if duplicates should be deleted
                                             #did this with a few parks. all have multiple repeats

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
40,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,33592.003207,NaN
288,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,4675.104705,NaN
598,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,56081.504073,NaN
883,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,67465.202576,NaN
1133,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,66697.446865,NaN
1413,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,37980.277154,NaN
1703,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,44842.076419,NaN
1975,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,38060.316661,NaN
2298,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,36887.598446,NaN
2510,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,38493.083105,NaN


In [15]:
parks_df = parks_df.drop_duplicates(subset = 'name', keep = 'first')  #dropping duplicate name and keeping first occurence
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN
2528,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN
2535,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN
2590,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN


In [19]:
parks_df = parks_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN
347,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN
348,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN
349,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN


In [17]:
parks_df[305:350]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
305,SkGinj_BiRFbY6QQ-Ln9rA,rock-creek-park-lewisburg,Rock Creek Park,https://s3-media2.fl.yelpcdn.com/bphoto/ZVVBW0...,False,https://www.yelp.com/biz/rock-creek-park-lewis...,1,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 35.453433112927, 'longitude': -86...",[],"{'address1': '101 Old Farmington Rd', 'address...",+19313591544,(931) 359-1544,32122.512354,NaN
306,zT6ayvRBuGF7OYPQA4dDYg,tims-ford-state-park-winchester-2,Tims Ford State Park,https://s3-media1.fl.yelpcdn.com/bphoto/jJKA65...,False,https://www.yelp.com/biz/tims-ford-state-park-...,13,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 35.219827, 'longitude': -86.255447}",[],"{'address1': '570 Tims Ford Dr', 'address2': N...",+19319683536,(931) 968-3536,86492.203190,NaN
307,JTKl-jQK76k5Cmdfd0AknQ,lewisburg-parks-rec-and-fitness-lewisburg,Lewisburg Parks Rec and Fitness,https://s3-media2.fl.yelpcdn.com/bphoto/r-GNkl...,False,https://www.yelp.com/biz/lewisburg-parks-rec-a...,2,"[{'alias': 'recreation', 'title': 'Recreation ...",4.5,"{'latitude': 35.4492545750614, 'longitude': -8...",[],"{'address1': '1551 Mooresville Hwy', 'address2...",+19313592482,(931) 359-2482,30481.554137,NaN
308,W8xHRMRMg8CqPh_IA6Eufg,mitzi-sweet-dog-park-lawrenceburg,Mitzi Sweet Dog Park,https://s3-media1.fl.yelpcdn.com/bphoto/D5wQt4...,False,https://www.yelp.com/biz/mitzi-sweet-dog-park-...,2,"[{'alias': 'dog_parks', 'title': 'Dog Parks'}]",4.5,"{'latitude': 35.2354362, 'longitude': -87.3469...",[],"{'address1': '830 Gaither Ave', 'address2': ''...",+19312423511,(931) 242-3511,50674.508021,NaN
309,Q6F9d42fBE-OeUl2j4PHMQ,fisherman-s-park-shelbyville,Fisherman’s Park,https://s3-media2.fl.yelpcdn.com/bphoto/ueWJN1...,False,https://www.yelp.com/biz/fisherman-s-park-shel...,1,"[{'alias': 'parks', 'title': 'Parks'}]",3.0,"{'latitude': 35.4816775, 'longitude': -86.4635...",[],"{'address1': '213 S Cannon Blvd', 'address2': ...",,,57122.814905,NaN
310,4vin4K9ROcmuKP8a9UG1Fg,wells-hill-park-fayetteville,Wells Hill Park,,False,https://www.yelp.com/biz/wells-hill-park-fayet...,1,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 35.09982755799566, 'longitude': -...",[],"{'address1': '91 Wells Hill Rd', 'address2': '...",,,73735.263967,NaN
311,1P9uLnhM4Uz6Vy4B6O9P9g,shelbyville-recreation-center-shelbyville,Shelbyville Recreation Center,https://s3-media2.fl.yelpcdn.com/bphoto/Wnt5VF...,False,https://www.yelp.com/biz/shelbyville-recreatio...,3,"[{'alias': 'amateursportsteams', 'title': 'Ama...",3.0,"{'latitude': 35.5037384, 'longitude': -86.4470...",[],"{'address1': '220 Tulip Tree Rd', 'address2': ...",+19316849780,(931) 684-9780,57881.098453,NaN
312,-ft9ygDBOLB9V8Xwh5JmRA,griffin-hv-park-shelbyville,Griffin Hv Park,https://s3-media4.fl.yelpcdn.com/bphoto/70CqC4...,False,https://www.yelp.com/biz/griffin-hv-park-shelb...,1,"[{'alias': 'amateursportsteams', 'title': 'Ama...",3.0,"{'latitude': 35.5037384, 'longitude': -86.4470...",[],"{'address1': '220 Tulip Tree Rd', 'address2': ...",+19316849780,(931) 684-9780,57881.098453,NaN
313,aWcqZVYPJ8P_wqS7E5jibg,pennyrile-state-park-beach-dawson-springs,Pennyrile state park beach,https://s3-media2.fl.yelpcdn.com/bphoto/La3KQc...,False,https://www.yelp.com/biz/pennyrile-state-park-...,4,"[{'alias': 'beaches', 'title': 'Beaches'}]",4.5,"{'latitude': 37.0716016774478, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,70908.810817,NaN
314,FOd8ZkUWp7W9A8YQMJDywg,mineral-mound-state-park-eddyville,Mineral Mound State Park,https://s3-media4.fl.yelpcdn.com/bphoto/TzuJ_N...,False,https://www.yelp.com/biz/mineral-mound-state-p...,3,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 37.0637093, 'longitude': -88.086113}",[],"{'address1': 'Mineral Mound Street Park', 'add...",+12703883673,(270) 388-3673,90924.484450,NaN


In [20]:
parks_df = parks_df.drop([27,61,69,86,98,112,117,121,127,133,145,149,150,152,161,163,175,176,183,185,186,192,194,201,203,208,210,215,217,222,229,230,237,239,243,254,255,257,258,259,260,261,266,267,270,271,272,281,285,287,296,297,298,299,300,301,302,313,315,316,320,330,334,341,342,345])
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN
4,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN
347,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN
348,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN
349,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN


In [22]:
parks_df = parks_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,category
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN,park
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN,park
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN,park
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN,park
4,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511,NaN,park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN,park
281,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN,park
282,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN,park
283,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN,park


In [23]:
parks_df['category'] = 'park'
parks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,category
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709,NaN,park
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736,NaN,park
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678,NaN,park
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456,NaN,park
4,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511,NaN,park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,UYEcvVgNhgkbUt4Z7p12WQ,cummins-falls-state-park-cookeville,Cummins Falls State Park,https://s3-media2.fl.yelpcdn.com/bphoto/SxT4ho...,False,https://www.yelp.com/biz/cummins-falls-state-p...,84,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.253729, 'longitude': -85.56541}",[],"{'address1': '390 Cummins Falls Ln', 'address2...",+19312687223,(931) 268-7223,67337.951515,NaN,park
281,7GfuDk7KLOKorBO2qWi7KA,burgess-falls-state-park-sparta,Burgess Falls State Park,https://s3-media1.fl.yelpcdn.com/bphoto/tLPs6w...,False,https://www.yelp.com/biz/burgess-falls-state-p...,53,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.01357104300603, 'longitude': -...",[],"{'address1': '4000 Burgess Falls Dr', 'address...",+19314325312,(931) 432-5312,69624.185315,NaN,park
282,HtWDageMy-cNd07cBWQSeQ,dogwood-park-cookeville,Dogwood park,https://s3-media2.fl.yelpcdn.com/bphoto/weRvOr...,False,https://www.yelp.com/biz/dogwood-park-cookevil...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.16433127651388, 'longitude': -...",[],"{'address1': '30 E Broad St', 'address2': '', ...",+19315204386,(931) 520-4386,72050.522374,NaN,park
283,5K_LErcS6RruRR1o0iKIjQ,cane-creek-park-cookeville,Cane Creek Park,,False,https://www.yelp.com/biz/cane-creek-park-cooke...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1611979931865, 'longitude': -8...",[],"{'address1': '45 E Broad St', 'address2': None...",+19315269591,(931) 526-9591,68802.462858,NaN,park


In [24]:
parks_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_parks.csv')

In [25]:
#now will repeat similar process to gather other activites for families in the 10 counties

In [26]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
museums = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'museums', location = location, search_limit = 20, offset = x)
        museums.append(response)
        print(len(museums))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [27]:
museums_df = pd.DataFrame()

for museum in range(len(museums)):
    df = pd.DataFrame(museums[museum]['businesses'])
    museums_df = museums_df.append(df)
    print(museums_df.shape)

(20, 15)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(120, 16)
(140, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(142, 16)
(162, 16)
(182, 16)
(202, 16)
(222, 16)
(242, 16)
(262, 16)
(282, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(283, 16)
(303, 16)
(323, 16)
(343, 16)
(363, 16)
(383, 16)
(403, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(415, 16)
(435, 16)
(455, 16)
(475, 16)
(495, 16)
(515, 16)
(535, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(547, 16)
(567

In [29]:
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
3,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,4kwny3ZC2QWyzhWLGhry-A,grinders-switch-winery-nashville,Grinder's Switch Winery,https://s3-media4.fl.yelpcdn.com/bphoto/nZnxYi...,False,https://www.yelp.com/biz/grinders-switch-winer...,99,"[{'alias': 'wineries', 'title': 'Wineries'}]",4.5,"{'latitude': 36.1645164489746, 'longitude': -8...",[],"{'address1': '1310 Clinton St', 'address2': 'S...",+16156790646,(615) 679-0646,44064.599189,$
7,mELlR_cfdqU6z9LYDbibMw,visitor-information-center-nashville,Visitor Information Center,https://s3-media4.fl.yelpcdn.com/bphoto/stPQ8R...,False,https://www.yelp.com/biz/visitor-information-c...,16,"[{'alias': 'visitorcenters', 'title': 'Visitor...",4.5,"{'latitude': 36.1591339111328, 'longitude': -8...",[],"{'address1': '501 Broadway', 'address2': '', '...",+16152594700,(615) 259-4700,42436.144406,NaN
8,ywcy56lRPyv6lx05w8Zr7g,city-of-nashville-nashville-2,City of Nashville,https://s3-media2.fl.yelpcdn.com/bphoto/dOppmq...,False,https://www.yelp.com/biz/city-of-nashville-nas...,37,"[{'alias': 'municipality', 'title': 'Municipal...",3.5,"{'latitude': 36.1653633117676, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': '...",,,42663.612699,NaN
9,Xnp_mwco11OL_mAp4IBwaA,picture-frame-warehouse-nashville-4,Picture Frame Warehouse,https://s3-media1.fl.yelpcdn.com/bphoto/5JUZiz...,False,https://www.yelp.com/biz/picture-frame-warehou...,13,"[{'alias': 'framing', 'title': 'Framing'}, {'a...",3.5,"{'latitude': 36.09439865389617, 'longitude': -...",[],"{'address1': '4320 Kenilwood Dr', 'address2': ...",+16153331161,(615) 333-1161,41482.530900,$$$


In [30]:
museums_df = museums_df.reset_index(drop=True)   
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
3,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,4kwny3ZC2QWyzhWLGhry-A,grinders-switch-winery-nashville,Grinder's Switch Winery,https://s3-media4.fl.yelpcdn.com/bphoto/nZnxYi...,False,https://www.yelp.com/biz/grinders-switch-winer...,99,"[{'alias': 'wineries', 'title': 'Wineries'}]",4.5,"{'latitude': 36.1645164489746, 'longitude': -8...",[],"{'address1': '1310 Clinton St', 'address2': 'S...",+16156790646,(615) 679-0646,44064.599189,$
1354,mELlR_cfdqU6z9LYDbibMw,visitor-information-center-nashville,Visitor Information Center,https://s3-media4.fl.yelpcdn.com/bphoto/stPQ8R...,False,https://www.yelp.com/biz/visitor-information-c...,16,"[{'alias': 'visitorcenters', 'title': 'Visitor...",4.5,"{'latitude': 36.1591339111328, 'longitude': -8...",[],"{'address1': '501 Broadway', 'address2': '', '...",+16152594700,(615) 259-4700,42436.144406,NaN
1355,ywcy56lRPyv6lx05w8Zr7g,city-of-nashville-nashville-2,City of Nashville,https://s3-media2.fl.yelpcdn.com/bphoto/dOppmq...,False,https://www.yelp.com/biz/city-of-nashville-nas...,37,"[{'alias': 'municipality', 'title': 'Municipal...",3.5,"{'latitude': 36.1653633117676, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': '...",,,42663.612699,NaN
1356,Xnp_mwco11OL_mAp4IBwaA,picture-frame-warehouse-nashville-4,Picture Frame Warehouse,https://s3-media1.fl.yelpcdn.com/bphoto/5JUZiz...,False,https://www.yelp.com/biz/picture-frame-warehou...,13,"[{'alias': 'framing', 'title': 'Framing'}, {'a...",3.5,"{'latitude': 36.09439865389617, 'longitude': -...",[],"{'address1': '4320 Kenilwood Dr', 'address2': ...",+16153331161,(615) 333-1161,41482.530900,$$$


In [31]:
museums_df[museums_df.duplicated(subset = 'name', keep = 'first')]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
142,mnwTbHyIgpMvOG9705zqWA,country-music-hall-of-fame-and-museum-nashville-3,Country Music Hall of Fame and Museum,https://s3-media3.fl.yelpcdn.com/bphoto/lr2eAA...,False,https://www.yelp.com/biz/country-music-hall-of...,688,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.158266, 'longitude': -86.776126}",[],"{'address1': '222 Rep. John Lewis Way S', 'add...",+16154162001,(615) 416-2001,3153.820124,NaN
143,z2FRDJSxRR3mmC9syR25xA,the-parthenon-nashville,The Parthenon,https://s3-media2.fl.yelpcdn.com/bphoto/juyVpf...,False,https://www.yelp.com/biz/the-parthenon-nashvil...,476,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",4.0,"{'latitude': 36.14967469964605, 'longitude': -...",[],"{'address1': '2500 West End Ave', 'address2': ...",+16158628431,(615) 862-8431,4830.218646,NaN
144,z87SlFdP6vZCjfjPkI-edQ,national-museum-of-african-american-music-nash...,National Museum of African American Music,https://s3-media2.fl.yelpcdn.com/bphoto/ZfjG4_...,False,https://www.yelp.com/biz/national-museum-of-af...,72,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16053057546768, 'longitude': -...",[],"{'address1': '510 Broadway', 'address2': '', '...",+16153018724,(615) 301-8724,2862.000472,NaN
145,jSYu2apG1uYq3eNzSA-OHw,johnny-cash-museum-nashville-4,Johnny Cash Museum,https://s3-media2.fl.yelpcdn.com/bphoto/qkanvY...,False,https://www.yelp.com/biz/johnny-cash-museum-na...,432,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.16084, 'longitude': -86.77586}",[],"{'address1': '119 3rd Ave S', 'address2': '', ...",+16152561777,(615) 256-1777,2877.143845,NaN
146,gNZC4qA7eMLQpUMeCYPAoA,frist-art-museum-nashville,Frist Art Museum,https://s3-media2.fl.yelpcdn.com/bphoto/EOAggj...,False,https://www.yelp.com/biz/frist-art-museum-nash...,230,"[{'alias': 'artmuseums', 'title': 'Art Museums'}]",4.0,"{'latitude': 36.157657, 'longitude': -86.783619}",[],"{'address1': '919 Broadway', 'address2': '', '...",+16152443340,(615) 244-3340,3146.369231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,4kwny3ZC2QWyzhWLGhry-A,grinders-switch-winery-nashville,Grinder's Switch Winery,https://s3-media4.fl.yelpcdn.com/bphoto/nZnxYi...,False,https://www.yelp.com/biz/grinders-switch-winer...,99,"[{'alias': 'wineries', 'title': 'Wineries'}]",4.5,"{'latitude': 36.1645164489746, 'longitude': -8...",[],"{'address1': '1310 Clinton St', 'address2': 'S...",+16156790646,(615) 679-0646,44064.599189,$
1354,mELlR_cfdqU6z9LYDbibMw,visitor-information-center-nashville,Visitor Information Center,https://s3-media4.fl.yelpcdn.com/bphoto/stPQ8R...,False,https://www.yelp.com/biz/visitor-information-c...,16,"[{'alias': 'visitorcenters', 'title': 'Visitor...",4.5,"{'latitude': 36.1591339111328, 'longitude': -8...",[],"{'address1': '501 Broadway', 'address2': '', '...",+16152594700,(615) 259-4700,42436.144406,NaN
1355,ywcy56lRPyv6lx05w8Zr7g,city-of-nashville-nashville-2,City of Nashville,https://s3-media2.fl.yelpcdn.com/bphoto/dOppmq...,False,https://www.yelp.com/biz/city-of-nashville-nas...,37,"[{'alias': 'municipality', 'title': 'Municipal...",3.5,"{'latitude': 36.1653633117676, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': '...",,,42663.612699,NaN
1356,Xnp_mwco11OL_mAp4IBwaA,picture-frame-warehouse-nashville-4,Picture Frame Warehouse,https://s3-media1.fl.yelpcdn.com/bphoto/5JUZiz...,False,https://www.yelp.com/biz/picture-frame-warehou...,13,"[{'alias': 'framing', 'title': 'Framing'}, {'a...",3.5,"{'latitude': 36.09439865389617, 'longitude': -...",[],"{'address1': '4320 Kenilwood Dr', 'address2': ...",+16153331161,(615) 333-1161,41482.530900,$$$


In [32]:
museums_df = museums_df.drop_duplicates(subset = 'name', keep = 'first')  
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
3,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,N6BB-kevywd7Zt07E0RXEA,the-smoke-house-lodge-and-cabins-monteagle,The Smoke House Lodge & Cabins,https://s3-media2.fl.yelpcdn.com/bphoto/_FN36a...,False,https://www.yelp.com/biz/the-smoke-house-lodge...,111,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.0,"{'latitude': 35.235441, 'longitude': -85.852699}",[],"{'address1': '844 W Main St', 'address2': '', ...",+19319242091,(931) 924-2091,86367.245308,$$
960,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN
1236,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
1239,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN


In [33]:
museums_df = museums_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
3,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,N6BB-kevywd7Zt07E0RXEA,the-smoke-house-lodge-and-cabins-monteagle,The Smoke House Lodge & Cabins,https://s3-media2.fl.yelpcdn.com/bphoto/_FN36a...,False,https://www.yelp.com/biz/the-smoke-house-lodge...,111,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.0,"{'latitude': 35.235441, 'longitude': -85.852699}",[],"{'address1': '844 W Main St', 'address2': '', ...",+19319242091,(931) 924-2091,86367.245308,$$
174,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN
175,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
176,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN


In [34]:
museums_df = museums_df.drop([18,21,35,45,49,54,62,65,69,83,85,86,89,96,97,102,104,105,106,107,108,109,110,111,112,113,115,117,120,121,124,126,127,128,130,131,132,133,134,135,138,139,140,141,142,156,164,165,166,171,172,173])
museums_df
#removing all rows that are not museums

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
3,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,dtdOcQ-v92f2tuZvvi_paQ,monty-wanamakers-southern-museum-and-gallery-m...,Monty Wanamaker's Southern Museum & Gallery,https://s3-media1.fl.yelpcdn.com/bphoto/-YcjT5...,False,https://www.yelp.com/biz/monty-wanamakers-sout...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 35.6807518005371, 'longitude': -8...",[],"{'address1': '210 E Main St', 'address2': '', ...",+19315078102,(931) 507-8102,61908.000011,NaN
174,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN
175,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
176,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN


In [35]:
museums_df = museums_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN
2,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN
3,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,dtdOcQ-v92f2tuZvvi_paQ,monty-wanamakers-southern-museum-and-gallery-m...,Monty Wanamaker's Southern Museum & Gallery,https://s3-media1.fl.yelpcdn.com/bphoto/-YcjT5...,False,https://www.yelp.com/biz/monty-wanamakers-sout...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 35.6807518005371, 'longitude': -8...",[],"{'address1': '210 E Main St', 'address2': '', ...",+19315078102,(931) 507-8102,61908.000011,NaN
122,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN
123,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
124,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN


In [36]:
museums_df['category'] = 'museum/landmark'
museums_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,category
0,dh5w2vBT4PCPo5DcbrUZ2w,customs-house-museum-clarksville,Customs House Museum,https://s3-media4.fl.yelpcdn.com/bphoto/pT3fUq...,False,https://www.yelp.com/biz/customs-house-museum-...,19,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5261427723157, 'longitude': -8...",[],"{'address1': '200 S Second St', 'address2': ''...",+19316485780,(931) 648-5780,38791.900814,NaN,museum/landmark
1,zaeod7eUNy_pjPG6yL-H2Q,fort-defiance-clarksville,Fort Defiance,https://s3-media3.fl.yelpcdn.com/bphoto/4wN0Mv...,False,https://www.yelp.com/biz/fort-defiance-clarksv...,14,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.5402794, 'longitude': -87.3713...",[],"{'address1': '120 A St', 'address2': '', 'addr...",,,40742.264465,NaN,museum/landmark
2,XzUyn8Ypy4Lwv5WhQiEN3w,storytellers-hideaway-farm-and-museum-bon-aqua,Storytellers Hideaway Farm & Museum,https://s3-media2.fl.yelpcdn.com/bphoto/1IpEGx...,False,https://www.yelp.com/biz/storytellers-hideaway...,23,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.95339, 'longitude': -87.32702}",[],"{'address1': '9676 Old Hwy 46', 'address2': No...",+19319964336,(931) 996-4336,38735.909078,NaN,museum/landmark
3,v0k0clT4nPvWo4GdQsfnxg,the-governor-frank-g-clement-railroad-hotel-mu...,The Governor Frank G Clement Railroad Hotel Mu...,https://s3-media4.fl.yelpcdn.com/bphoto/V1kiwe...,False,https://www.yelp.com/biz/the-governor-frank-g-...,2,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.0,"{'latitude': 36.0807724, 'longitude': -87.3577...",[],"{'address1': '100 Frank Clement Pl', 'address2...",+16154460500,(615) 446-0500,32515.937369,NaN,museum/landmark
4,y_nuIBABDIZqKE3LedJioQ,don-f-pratt-museum-fort-campbell,Don F Pratt Museum,https://s3-media4.fl.yelpcdn.com/bphoto/yWyLkS...,False,https://www.yelp.com/biz/don-f-pratt-museum-fo...,6,"[{'alias': 'museums', 'title': 'Museums'}]",4.5,"{'latitude': 36.6411936, 'longitude': -87.4567...",[],"{'address1': '5702 Tennessee Ave', 'address2':...",+12707984986,(270) 798-4986,53959.411693,NaN,museum/landmark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,dtdOcQ-v92f2tuZvvi_paQ,monty-wanamakers-southern-museum-and-gallery-m...,Monty Wanamaker's Southern Museum & Gallery,https://s3-media1.fl.yelpcdn.com/bphoto/-YcjT5...,False,https://www.yelp.com/biz/monty-wanamakers-sout...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 35.6807518005371, 'longitude': -8...",[],"{'address1': '210 E Main St', 'address2': '', ...",+19315078102,(931) 507-8102,61908.000011,NaN,museum/landmark
122,3B7sd1IplPrxdEkHNOC8ag,south-central-kentucky-cultural-center-glasgow,South Central Kentucky Cultural Center,https://s3-media3.fl.yelpcdn.com/bphoto/XzXG94...,False,https://www.yelp.com/biz/south-central-kentuck...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.996957, 'longitude': -85.912694}",[],"{'address1': '200 W Water St', 'address2': '',...",+12706519792,(270) 651-9792,79276.910092,NaN,museum/landmark
123,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN,museum/landmark
124,BGhgxzyl4ECtYBzlU8iG3g,cookeville-childrens-museum-cookeville,Cookeville Childrens Museum,https://s3-media4.fl.yelpcdn.com/bphoto/zW3gER...,False,https://www.yelp.com/biz/cookeville-childrens-...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.16605, 'longitude': -85.5072403}",[],"{'address1': '36 W 2nd St', 'address2': None, ...",+19319797529,(931) 979-7529,71703.003792,NaN

In [37]:
museums_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_museums.csv')

In [ ]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
landmarks = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'landmarks', location = location, search_limit = 20, offset = x)
        landmarks.append(response)
        print(len(landmarks))

In [ ]:
landmarks_df = pd.DataFrame()

for landmark in range(len(landmarks)):
    df = pd.DataFrame(landmarks[landmark]['businesses'])
    landmarks_df = landmarks_df.append(df)
    print(landmarks_df.shape)

In [ ]:
landmarks_df = landmarks_df.reset_index(drop=True)   
landmarks_df

#seeing that most rows are not landmarks and have already been grabbed in museum search

In [38]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
libraries = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'libraries', location = location, search_limit = 20, offset = x)
        libraries.append(response)
        print(len(libraries))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [39]:
libraries_df = pd.DataFrame()

for library in range(len(libraries)):
    df = pd.DataFrame(libraries[library]['businesses'])
    libraries_df = libraries_df.append(df)
    print(libraries_df.shape)

(20, 15)
(40, 15)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(55, 16)
(75, 16)
(95, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(109, 16)
(129, 16)
(149, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(155, 16)
(175, 16)
(195, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(201, 16)
(221, 16)
(241, 16)
(245, 1

In [40]:
libraries_df = libraries_df.drop_duplicates(subset = 'name', keep = 'first')  
libraries_df = libraries_df.reset_index(drop=True)  
libraries_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,cjIbKTknnvvw8E7FO6SCzw,nashville-public-library-nashville-2,Nashville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/oEwtcu...,False,https://www.yelp.com/biz/nashville-public-libr...,87,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.161758, 'longitude': -86.781608}",[],"{'address1': '615 Church St', 'address2': '', ...",+16158625800,(615) 862-5800,29867.224261,NaN
1,VCDIfeiT3xzWR--eyJk8_g,nashville-public-library-green-hills-branch-na...,Nashville Public Library - Green Hills Branch,https://s3-media1.fl.yelpcdn.com/bphoto/dTb9Zg...,False,https://www.yelp.com/biz/nashville-public-libr...,20,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 36.109909, 'longitude': -86.809107}",[],"{'address1': '3701 Benham Ave', 'address2': ''...",+16158625863,(615) 862-5863,30046.783562,NaN
2,Cs7hHsWMLvMxtxwDsRJCGw,nashville-public-library-bellevue-branch-nashv...,Nashville Public Library - Bellevue Branch,https://s3-media2.fl.yelpcdn.com/bphoto/tVkhdT...,False,https://www.yelp.com/biz/nashville-public-libr...,12,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.0691351443529, 'longitude': -8...",[],"{'address1': '720 Baugh Rd', 'address2': '', '...",+16158625854,(615) 862-5854,24538.844400,NaN
3,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,15,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN
4,71-dWjMS7aBUc0oG4wdsww,spring-hill-library-spring-hill,Spring Hill Library,https://s3-media3.fl.yelpcdn.com/bphoto/4ER2qm...,False,https://www.yelp.com/biz/spring-hill-library-s...,2,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.74972, 'longitude': -86.938}",[],"{'address1': '144 Kedron Pkwy', 'address2': ''...",+19314862932,(931) 486-2932,57236.245050,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,vqsCccA6jfcH34muTD1uHA,city-of-bowling-green-bowling-green-2,City of Bowling Green,https://s3-media4.fl.yelpcdn.com/bphoto/t5PJsg...,False,https://www.yelp.com/biz/city-of-bowling-green...,1,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",5.0,"{'latitude': 36.9924079, 'longitude': -86.4438...",[],"{'address1': '1001 College St', 'address2': ''...",+12703933000,(270) 393-3000,66014.808955,NaN
61,gjibOZdKcD1t8cFRg4Ak5A,coffee-county-lannom-memorial-library-tullahoma,Coffee County Lannom Memorial Library,,False,https://www.yelp.com/biz/coffee-county-lannom-...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.3608638, 'longitude': -86.2192...",[],"{'address1': '312 North Collins St', 'address2...",+19314552460,(931) 455-2460,58364.385353,NaN
62,HoLD1c5ivg_zMxGjHYCxgA,manchester-branch-library-manchester,Manchester Branch Library,,False,https://www.yelp.com/biz/manchester-branch-lib...,3,"[{'alias': 'libraries', 'title': 'Libraries'}]",2.0,"{'latitude': 35.47384, 'longitude': -86.07763}",[],"{'address1': '1005 Hillsboro Blvd', 'address2'...",+19317235143,(931) 723-5143,52928.010680,NaN
63,YwU6Rrrx5JgU2w8UVjeFyQ,morrison-library-morrison,Morrison Library,,False,https://www.yelp.com/biz/morrison-library-morr...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",1.0,"{'latitude': 35.60276, 'longitude': -85.91965}",[],"{'address1': '100 W Maple St', 'address2': '',...",+19316352060,(931) 635-2060,53513.114447,NaN


In [41]:
libraries_df[30:66]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
30,wBxpUYVxGJ7bLXQi7WA1Iw,gallatin-public-library-gallatin,Gallatin Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/ftKVLf...,False,https://www.yelp.com/biz/gallatin-public-libra...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.38796, 'longitude': -86.44522}",[],"{'address1': '123 E Main St', 'address2': '', ...",+16154521722,(615) 452-1722,60415.521683,NaN
31,_PimUuB2peKeBdwxOUz9Tw,nashville-public-library-thompson-lane-branch-...,Nashville Public Library - Thompson Lane Branch,https://s3-media1.fl.yelpcdn.com/bphoto/35MWax...,False,https://www.yelp.com/biz/nashville-public-libr...,5,"[{'alias': 'libraries', 'title': 'Libraries'}]",3.0,"{'latitude': 36.1111019, 'longitude': -86.7432...",[],"{'address1': '380 Thompson Ln', 'address2': ''...",+16158625873,(615) 862-5873,35198.026180,NaN
32,DJvK9VhSHJi5KB1G0sFm7Q,hopkinsville-christian-county-public-library-h...,Hopkinsville-Christian County Public Library,https://s3-media1.fl.yelpcdn.com/bphoto/LPSGd9...,False,https://www.yelp.com/biz/hopkinsville-christia...,2,"[{'alias': 'libraries', 'title': 'Libraries'}]",2.5,"{'latitude': 36.8652547, 'longitude': -87.490576}",[],"{'address1': '1101 Bethel St', 'address2': '',...",+12708874262,(270) 887-4262,77120.080047,NaN
33,nK9hqSvELXLmB23-unafCg,library-public-library-of-nashville-and-dvdsn-...,Library-Public Library of Nashville and Dvdsn ...,https://s3-media3.fl.yelpcdn.com/bphoto/IniH7R...,False,https://www.yelp.com/biz/library-public-librar...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 36.168342, 'longitude': -86.683424}",[],"{'address1': '2315 Lebanon Rd', 'address2': ''...",+16158625750,(615) 862-5750,38136.669712,NaN
34,DYTzbh1rg4Pl83MSRWlcXw,technology-engagement-center-murfreesboro,Technology Engagement Center,,False,https://www.yelp.com/biz/technology-engagement...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.83918, 'longitude': -86.36499}",[],"{'address1': '306 Minerva Dr', 'address2': '',...",+16152258312,(615) 225-8312,79956.014982,NaN
35,LTDfFtXLCQVnVu8NKHsbCg,gorham-macbane-public-library-springfield,Gorham-MacBane Public Library,,False,https://www.yelp.com/biz/gorham-macbane-public...,1,"[{'alias': 'libraries', 'title': 'Libraries'}]",1.0,"{'latitude': 36.5109888464212, 'longitude': -8...",[],"{'address1': '405 White St', 'address2': '', '...",+16153845123,(615) 384-5123,34581.819549,NaN
36,kdQMqkD5VxVelrnQ8--wXg,southeast-regional-community-center-antioch,Southeast Regional Community Center,https://s3-media2.fl.yelpcdn.com/bphoto/fr_chT...,False,https://www.yelp.com/biz/southeast-regional-co...,4,"[{'alias': 'communitycenters', 'title': 'Commu...",4.0,"{'latitude': 36.0523764544461, 'longitude': -8...",[],"{'address1': '5260 Hickory Hollow Pkwy', 'addr...",+16158628902,(615) 862-8902,45162.279725,NaN
37,hvOWbFacsVZwFIAAXtN3BA,cheekwood-nashville,Cheekwood,https://s3-media1.fl.yelpcdn.com/bphoto/pZCkb4...,False,https://www.yelp.com/biz/cheekwood-nashville?a...,296,"[{'alias': 'gardens', 'title': 'Botanical Gard...",4.0,"{'latitude': 36.0881153743368, 'longitude': -8...",[],"{'address1': '1200 Forrest Park Dr', 'address2...",+16153568000,(615) 356-8000,26633.610597,NaN
38,VHLuNAXhYeaEFEm_rsKiqQ,volunteer-state-community-college-gallatin,Volunteer State Community College,https://s3-media1.fl.yelpcdn.com/bphoto/6VuFLo...,False,https://www.yelp.com/biz/volunteer-state-commu...,1,"[{'alias': 'collegeuniv', 'title': 'Colleges &...",5.0,"{'latitude': 36.36236425003344, 'longitude': -...",[],"{'address1': '1480 Nashville Pike', 'address2'...",+16154528600,(615) 452-8600,55213.655839,NaN
39,SbYP1hRfLShz13MpN58c2Q,middle-tennessee-state-university-murfreesboro,Middle Tennessee State University,https://s3-media4.fl.yelpcdn.com/bphoto/JxUSLV...,False,https://www.yelp.com/biz/middle-tennessee-stat...,26,"[{'alia

In [42]:
libraries_df = libraries_df.drop([36,37,38,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,60])
libraries_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,cjIbKTknnvvw8E7FO6SCzw,nashville-public-library-nashville-2,Nashville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/oEwtcu...,False,https://www.yelp.com/biz/nashville-public-libr...,87,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.161758, 'longitude': -86.781608}",[],"{'address1': '615 Church St', 'address2': '', ...",+16158625800,(615) 862-5800,29867.224261,NaN
1,VCDIfeiT3xzWR--eyJk8_g,nashville-public-library-green-hills-branch-na...,Nashville Public Library - Green Hills Branch,https://s3-media1.fl.yelpcdn.com/bphoto/dTb9Zg...,False,https://www.yelp.com/biz/nashville-public-libr...,20,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 36.109909, 'longitude': -86.809107}",[],"{'address1': '3701 Benham Ave', 'address2': ''...",+16158625863,(615) 862-5863,30046.783562,NaN
2,Cs7hHsWMLvMxtxwDsRJCGw,nashville-public-library-bellevue-branch-nashv...,Nashville Public Library - Bellevue Branch,https://s3-media2.fl.yelpcdn.com/bphoto/tVkhdT...,False,https://www.yelp.com/biz/nashville-public-libr...,12,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.0691351443529, 'longitude': -8...",[],"{'address1': '720 Baugh Rd', 'address2': '', '...",+16158625854,(615) 862-5854,24538.844400,NaN
3,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,15,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN
4,71-dWjMS7aBUc0oG4wdsww,spring-hill-library-spring-hill,Spring Hill Library,https://s3-media3.fl.yelpcdn.com/bphoto/4ER2qm...,False,https://www.yelp.com/biz/spring-hill-library-s...,2,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.74972, 'longitude': -86.938}",[],"{'address1': '144 Kedron Pkwy', 'address2': ''...",+19314862932,(931) 486-2932,57236.245050,NaN
5,N-m8jfnawwenfL3-jzlbqg,the-brentwood-library-brentwood,The Brentwood Library,https://s3-media1.fl.yelpcdn.com/bphoto/vCB4FD...,False,https://www.yelp.com/biz/the-brentwood-library...,16,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.9945549, 'longitude': -86.7895...",[],"{'address1': '8109 Concord Rd', 'address2': ''...",+16153710090,(615) 371-0090,39444.436542,NaN
6,cqvGrxn4rQwX7j2OyyrulQ,robert-f-sink-memorial-library-clarksville,Robert F Sink Memorial Library,https://s3-media3.fl.yelpcdn.com/bphoto/S5ev3b...,False,https://www.yelp.com/biz/robert-f-sink-memoria...,4,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.6380657142469, 'longitude': -8...",[],"{'address1': '38 Screaming Eagle Blvd', 'addre...",+12707985729,(270) 798-5729,53568.694701,NaN
7,d0CDRKPdm1AT-WKQU4JmVA,williamson-county-public-library-franklin,Williamson County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/gEucRB...,False,https://www.yelp.com/biz/williamson-county-pub...,8,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.9152088173186, 'longitude': -8...",[],"{'address1': '1314 Columbia Ave', 'address2': ...",+16155951243,(615) 595-1243,42089.737179,NaN
8,XjEF_0Rg7HgV0dKGlV30YQ,nolensville-public-library-nolensville,Nolensville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/XTQBlW...,False,https://www.yelp.com/biz/nolensville-public-li...,4,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.962043, 'longitude': -86.673065}",[],"{'address1': '915 Oldham Dr', 'address2': '', ...",+16157765490,(615) 776-5490,49602.378555,NaN
9,pUmBA1JOfOMy_ymSov1b7A,nashville-public-library-richland-park-branch-...,Nashville Public Library - Richland Park Branch,https://s3-media4.fl.yelpcdn.com/

In [43]:
libraries_df = libraries_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
libraries_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,cjIbKTknnvvw8E7FO6SCzw,nashville-public-library-nashville-2,Nashville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/oEwtcu...,False,https://www.yelp.com/biz/nashville-public-libr...,87,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.161758, 'longitude': -86.781608}",[],"{'address1': '615 Church St', 'address2': '', ...",+16158625800,(615) 862-5800,29867.224261,NaN
1,VCDIfeiT3xzWR--eyJk8_g,nashville-public-library-green-hills-branch-na...,Nashville Public Library - Green Hills Branch,https://s3-media1.fl.yelpcdn.com/bphoto/dTb9Zg...,False,https://www.yelp.com/biz/nashville-public-libr...,20,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 36.109909, 'longitude': -86.809107}",[],"{'address1': '3701 Benham Ave', 'address2': ''...",+16158625863,(615) 862-5863,30046.783562,NaN
2,Cs7hHsWMLvMxtxwDsRJCGw,nashville-public-library-bellevue-branch-nashv...,Nashville Public Library - Bellevue Branch,https://s3-media2.fl.yelpcdn.com/bphoto/tVkhdT...,False,https://www.yelp.com/biz/nashville-public-libr...,12,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.0691351443529, 'longitude': -8...",[],"{'address1': '720 Baugh Rd', 'address2': '', '...",+16158625854,(615) 862-5854,24538.844400,NaN
3,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,15,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN
4,71-dWjMS7aBUc0oG4wdsww,spring-hill-library-spring-hill,Spring Hill Library,https://s3-media3.fl.yelpcdn.com/bphoto/4ER2qm...,False,https://www.yelp.com/biz/spring-hill-library-s...,2,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.74972, 'longitude': -86.938}",[],"{'address1': '144 Kedron Pkwy', 'address2': ''...",+19314862932,(931) 486-2932,57236.245050,NaN
5,N-m8jfnawwenfL3-jzlbqg,the-brentwood-library-brentwood,The Brentwood Library,https://s3-media1.fl.yelpcdn.com/bphoto/vCB4FD...,False,https://www.yelp.com/biz/the-brentwood-library...,16,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.9945549, 'longitude': -86.7895...",[],"{'address1': '8109 Concord Rd', 'address2': ''...",+16153710090,(615) 371-0090,39444.436542,NaN
6,cqvGrxn4rQwX7j2OyyrulQ,robert-f-sink-memorial-library-clarksville,Robert F Sink Memorial Library,https://s3-media3.fl.yelpcdn.com/bphoto/S5ev3b...,False,https://www.yelp.com/biz/robert-f-sink-memoria...,4,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.6380657142469, 'longitude': -8...",[],"{'address1': '38 Screaming Eagle Blvd', 'addre...",+12707985729,(270) 798-5729,53568.694701,NaN
7,d0CDRKPdm1AT-WKQU4JmVA,williamson-county-public-library-franklin,Williamson County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/gEucRB...,False,https://www.yelp.com/biz/williamson-county-pub...,8,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.9152088173186, 'longitude': -8...",[],"{'address1': '1314 Columbia Ave', 'address2': ...",+16155951243,(615) 595-1243,42089.737179,NaN
8,XjEF_0Rg7HgV0dKGlV30YQ,nolensville-public-library-nolensville,Nolensville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/XTQBlW...,False,https://www.yelp.com/biz/nolensville-public-li...,4,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.962043, 'longitude': -86.673065}",[],"{'address1': '915 Oldham Dr', 'address2': '', ...",+16157765490,(615) 776-5490,49602.378555,NaN
9,pUmBA1JOfOMy_ymSov1b7A,nashville-public-library-richland-park-branch-...,Nashville Public Library - Richland Park Branch,https://s3-media4.fl.yelpcdn.com/

In [44]:
libraries_df['category'] = 'library'
libraries_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,category
0,cjIbKTknnvvw8E7FO6SCzw,nashville-public-library-nashville-2,Nashville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/oEwtcu...,False,https://www.yelp.com/biz/nashville-public-libr...,87,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.161758, 'longitude': -86.781608}",[],"{'address1': '615 Church St', 'address2': '', ...",+16158625800,(615) 862-5800,29867.224261,NaN,library
1,VCDIfeiT3xzWR--eyJk8_g,nashville-public-library-green-hills-branch-na...,Nashville Public Library - Green Hills Branch,https://s3-media1.fl.yelpcdn.com/bphoto/dTb9Zg...,False,https://www.yelp.com/biz/nashville-public-libr...,20,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 36.109909, 'longitude': -86.809107}",[],"{'address1': '3701 Benham Ave', 'address2': ''...",+16158625863,(615) 862-5863,30046.783562,NaN,library
2,Cs7hHsWMLvMxtxwDsRJCGw,nashville-public-library-bellevue-branch-nashv...,Nashville Public Library - Bellevue Branch,https://s3-media2.fl.yelpcdn.com/bphoto/tVkhdT...,False,https://www.yelp.com/biz/nashville-public-libr...,12,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.0691351443529, 'longitude': -8...",[],"{'address1': '720 Baugh Rd', 'address2': '', '...",+16158625854,(615) 862-5854,24538.844400,NaN,library
3,cAIIsVrYIlBm8M4HHKqOrA,clarksville-montgomery-county-public-library-c...,Clarksville Montgomery County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/Vj0Epj...,False,https://www.yelp.com/biz/clarksville-montgomer...,15,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 36.521023, 'longitude': -87.340736}",[],"{'address1': '350 Pageant Ln', 'address2': 'St...",+19316488826,(931) 648-8826,37393.358988,NaN,library
4,71-dWjMS7aBUc0oG4wdsww,spring-hill-library-spring-hill,Spring Hill Library,https://s3-media3.fl.yelpcdn.com/bphoto/4ER2qm...,False,https://www.yelp.com/biz/spring-hill-library-s...,2,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.74972, 'longitude': -86.938}",[],"{'address1': '144 Kedron Pkwy', 'address2': ''...",+19314862932,(931) 486-2932,57236.245050,NaN,library
5,N-m8jfnawwenfL3-jzlbqg,the-brentwood-library-brentwood,The Brentwood Library,https://s3-media1.fl.yelpcdn.com/bphoto/vCB4FD...,False,https://www.yelp.com/biz/the-brentwood-library...,16,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.9945549, 'longitude': -86.7895...",[],"{'address1': '8109 Concord Rd', 'address2': ''...",+16153710090,(615) 371-0090,39444.436542,NaN,library
6,cqvGrxn4rQwX7j2OyyrulQ,robert-f-sink-memorial-library-clarksville,Robert F Sink Memorial Library,https://s3-media3.fl.yelpcdn.com/bphoto/S5ev3b...,False,https://www.yelp.com/biz/robert-f-sink-memoria...,4,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.5,"{'latitude': 36.6380657142469, 'longitude': -8...",[],"{'address1': '38 Screaming Eagle Blvd', 'addre...",+12707985729,(270) 798-5729,53568.694701,NaN,library
7,d0CDRKPdm1AT-WKQU4JmVA,williamson-county-public-library-franklin,Williamson County Public Library,https://s3-media2.fl.yelpcdn.com/bphoto/gEucRB...,False,https://www.yelp.com/biz/williamson-county-pub...,8,"[{'alias': 'libraries', 'title': 'Libraries'}]",4.0,"{'latitude': 35.9152088173186, 'longitude': -8...",[],"{'address1': '1314 Columbia Ave', 'address2': ...",+16155951243,(615) 595-1243,42089.737179,NaN,library
8,XjEF_0Rg7HgV0dKGlV30YQ,nolensville-public-library-nolensville,Nolensville Public Library,https://s3-media3.fl.yelpcdn.com/bphoto/XTQBlW...,False,https://www.yelp.com/biz/nolensville-public-li...,4,"[{'alias': 'libraries', 'title': 'Libraries'}]",5.0,"{'latitude': 35.962043, 'longitude': -86.673065}",[],"{'address1': '915 Oldham Dr', 'address2': '', ...",+16157765490,(615) 776-5490,49602.378555,NaN,library
9,pUmBA1JOfOMy_ymSov1b7A,nashville-public-library-richland-park-branch-...,

In [45]:
libraries_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_libraries.csv')

In [46]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
arcades = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'arcades', location = location, search_limit = 20, offset = x)
        arcades.append(response)
        print(len(arcades))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [47]:
arcades_df = pd.DataFrame()

for arcade in range(len(arcades)):
    df = pd.DataFrame(arcades[arcade]['businesses'])
    arcades_df = arcades_df.append(df)
    print(arcades_df.shape)

(20, 16)
(40, 16)
(60, 16)
(80, 16)
(100, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(107, 16)
(127, 16)
(147, 16)
(167, 16)
(187, 16)
(207, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(213, 16)
(233, 16)
(253, 16)
(273, 16)
(293, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(299, 16)
(319, 16)
(339, 16)
(359, 16)
(379, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(388, 16)
(408

In [48]:
arcades_df = arcades_df.drop_duplicates(subset = 'name', keep = 'first')  
arcades_df = arcades_df.reset_index(drop=True)  
arcades_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,UiURMNx6i-qeBWvwcIKPgQ,joyfull-arcade-spring-hill-2,Joyfull Arcade,https://s3-media2.fl.yelpcdn.com/bphoto/z5EZ5s...,False,https://www.yelp.com/biz/joyfull-arcade-spring...,1,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.77542748017264, 'longitude': -...",[],"{'address1': '2001 Campbell Station Pkwy', 'ad...",+16156141184,(615) 614-1184,55092.784846,NaN
1,8mEcWgU7q6lWchl-aJy_dw,the-city-forum-clarksville,The City Forum,https://s3-media3.fl.yelpcdn.com/bphoto/nLNur6...,False,https://www.yelp.com/biz/the-city-forum-clarks...,57,"[{'alias': 'mini_golf', 'title': 'Mini Golf'},...",4.0,"{'latitude': 36.511204, 'longitude': -87.2745814}",[],"{'address1': '2231-T Madison St', 'address2': ...",+19319195051,(931) 919-5051,33312.136770,NaN
2,3MuUUkRYnx56VAYyENvQDA,up-down-nashville-nashville-2,Up-Down Nashville,https://s3-media3.fl.yelpcdn.com/bphoto/JuPSXB...,False,https://www.yelp.com/biz/up-down-nashville-nas...,23,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 36.1759, 'longitude': -86.7546}","[pickup, delivery]","{'address1': '927 Woodland St', 'address2': No...",+16152263747,(615) 226-3747,31732.220217,$
3,_qPvx7G1-Q5I8JYuDTXlPQ,warrior-zone-fort-campbell,Warrior Zone,https://s3-media3.fl.yelpcdn.com/bphoto/KCnYC6...,False,https://www.yelp.com/biz/warrior-zone-fort-cam...,1,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.66417655503705, 'longitude': -...",[],"{'address1': '3910 Indiana Ave', 'address2': '...",+12704610683,(270) 461-0683,56965.972820,NaN
4,rRsh-zolgfN96fz6UU_uzA,magic-wheels-fun-center-clarksville,Magic Wheels Fun Center,https://s3-media3.fl.yelpcdn.com/bphoto/YE170t...,False,https://www.yelp.com/biz/magic-wheels-fun-cent...,16,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.0,"{'latitude': 36.58135790157406, 'longitude': -...",[],"{'address1': '1671 Fort Campbell Blvd', 'addre...",+19319067300,(931) 906-7300,46433.313553,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,ztQK4EfzwMbj3zzceiJxAA,tullahoma-bowling-lanes-tullahoma,Tullahoma Bowling Lanes,https://s3-media2.fl.yelpcdn.com/bphoto/iXGjte...,False,https://www.yelp.com/biz/tullahoma-bowling-lan...,7,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",2.5,"{'latitude': 35.3563, 'longitude': -86.19797}",[],"{'address1': '900 S Anderson St', 'address2': ...",+19314551947,(931) 455-1947,59492.132681,NaN
107,tHVupO1RIdd-DjZ-BbytWQ,ralphies-fun-center-glasgow,Ralphie's Fun Center,https://s3-media4.fl.yelpcdn.com/bphoto/yoZf_k...,False,https://www.yelp.com/biz/ralphies-fun-center-g...,9,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",4.5,"{'latitude': 37.0086209, 'longitude': -85.9189...",[],"{'address1': '702 Happy Valley Rd', 'address2'...",+12706294263,(270) 629-4263,79927.217044,NaN
108,rvimEgjXRq38ACp1wFXi_g,paradise-games-and-arcade-cookeville,Paradise Games and Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/dYpjpD...,False,https://www.yelp.com/biz/paradise-games-and-ar...,1,"[{'alias': 'videogamestores', 'title': 'Video ...",5.0,"{'latitude': 36.16233218340348, 'longitude': -...",[],"{'address1': '8 S Washington Ave', 'address2':...",+19312149263,(931) 214-9263,72450.315101,NaN
109,Jdx6joQwAE_byUINnYl1Jg,the-sk8-cookeville-2,The Sk8,https://s3-media1.fl.yelpcdn.com/bphoto/eH_2Mt...,False,https://www.yelp.com/biz/the-sk8-cookeville-2?...,4,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.5,"{'latitude': 36.1399541973511, 'longitude': -8...",[],"{'address1': '1810 Foreman Dr', 'address2': ''...",+19313722758,(931) 372-2758,69865.781632,NaN


In [49]:
arcades_df[100:112]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
100,W9eqygGVzaqRrsFkOBDo4Q,penalties-sports-bar-and-grill-shelbyville,Penalties Sports Bar & Grill,https://s3-media2.fl.yelpcdn.com/bphoto/zXqzAR...,False,https://www.yelp.com/biz/penalties-sports-bar-...,4,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.5,"{'latitude': 35.509797, 'longitude': -86.457254}",[],"{'address1': '1731 N Main St', 'address2': 'St...",+19317356495,(931) 735-6495,56708.412649,NaN
101,dRSMgDaCRtl7AYRDXmOcDQ,chuck-e-cheese-bowling-green,Chuck E Cheese,https://s3-media4.fl.yelpcdn.com/bphoto/1RtbEW...,False,https://www.yelp.com/biz/chuck-e-cheese-bowlin...,4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 36.94968, 'longitude': -86.427493}","[delivery, pickup]","{'address1': '2625 Scottsville Rd', 'address2'...",+12703931892,(270) 393-1892,63013.072466,NaN
102,8tQl6IOjb8R-qyxZFry6gg,detention-morrison,Detention,https://s3-media4.fl.yelpcdn.com/bphoto/qGJxU9...,False,https://www.yelp.com/biz/detention-morrison?ad...,1,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",5.0,"{'latitude': 35.5979563, 'longitude': -85.926836}",[],"{'address1': '615 School St', 'address2': None...",+19317737529,(931) 773-7529,53180.034787,NaN
103,xskxtMqyoPaPfIXoHKQkYA,evins-mill-smithville-2,Evins Mill,https://s3-media2.fl.yelpcdn.com/bphoto/hFeBlU...,False,https://www.yelp.com/biz/evins-mill-smithville...,27,"[{'alias': 'venues', 'title': 'Venues & Event ...",4.5,"{'latitude': 35.9438794562482, 'longitude': -8...",[],"{'address1': '1535 Evins Mill Rd', 'address2':...",+16152693740,(615) 269-3740,58535.861862,NaN
104,Kh2rxQanmUiVkP-ti6FDzA,kps-mcminnville-lanes-mcminnville,KP's McMinnville Lanes,https://s3-media1.fl.yelpcdn.com/bphoto/dDYLJT...,False,https://www.yelp.com/biz/kps-mcminnville-lanes...,3,"[{'alias': 'bowling', 'title': 'Bowling'}]",2.5,"{'latitude': 35.69977, 'longitude': -85.77723}",[],"{'address1': '200 Hobson St', 'address2': 'Ste...",+19314736679,(931) 473-6679,60772.762108,NaN
105,Y5842j6g2kBCD54JU7UcGQ,bonnaroo-festival-manchester,Bonnaroo Festival,https://s3-media2.fl.yelpcdn.com/bphoto/weqmEw...,False,https://www.yelp.com/biz/bonnaroo-festival-man...,78,"[{'alias': 'festivals', 'title': 'Festivals'}]",4.5,"{'latitude': 35.497066, 'longitude': -86.087617}",[],"{'address1': '700 Acre Farm In Manchester', 'a...",,,50311.533966,NaN
106,ztQK4EfzwMbj3zzceiJxAA,tullahoma-bowling-lanes-tullahoma,Tullahoma Bowling Lanes,https://s3-media2.fl.yelpcdn.com/bphoto/iXGjte...,False,https://www.yelp.com/biz/tullahoma-bowling-lan...,7,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",2.5,"{'latitude': 35.3563, 'longitude': -86.19797}",[],"{'address1': '900 S Anderson St', 'address2': ...",+19314551947,(931) 455-1947,59492.132681,NaN
107,tHVupO1RIdd-DjZ-BbytWQ,ralphies-fun-center-glasgow,Ralphie's Fun Center,https://s3-media4.fl.yelpcdn.com/bphoto/yoZf_k...,False,https://www.yelp.com/biz/ralphies-fun-center-g...,9,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",4.5,"{'latitude': 37.0086209, 'longitude': -85.9189...",[],"{'address1': '702 Happy Valley Rd', 'address2'...",+12706294263,(270) 629-4263,79927.217044,NaN
108,rvimEgjXRq38ACp1wFXi_g,paradise-games-and-arcade-cookeville,Paradise Games and Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/dYpjpD...,False,https://www.yelp.com/biz/paradise-games-and-ar...,1,"[{'alias': 'videogamestores', 'title': 'Video ...",5.0,"{'latitude': 36.16233218340348, 'longitude': -...",[],"{'address1': '8 S Washington Ave', 'address2':...",+19312149263,(931) 214-9263,72450.315101,NaN
109,Jdx6joQwAE_byUINnYl1Jg,the-sk8-cookeville-2,The Sk8,https://s3-media1.fl.yelpcdn.com/bphoto/eH_2Mt...,False,https://www.yelp.com/biz/the-sk8-cookeville-2?...,4,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.5,"{'latitude': 36.1399541973511, 'longitude': -8...",[],"{'address1': '1810 Foreman Dr', 'address2': ''...",+19313722758,(931) 3

In [50]:
arcades_df = arcades_df.drop([3,14,19,20,28,39,43,45,48,50,51,52,54,56,60,61,62,64,66,69,70,71,74,75,78,79,81,82,83,87,89,90,92,94,95,98,99,100,103,105])
arcades_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,UiURMNx6i-qeBWvwcIKPgQ,joyfull-arcade-spring-hill-2,Joyfull Arcade,https://s3-media2.fl.yelpcdn.com/bphoto/z5EZ5s...,False,https://www.yelp.com/biz/joyfull-arcade-spring...,1,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.77542748017264, 'longitude': -...",[],"{'address1': '2001 Campbell Station Pkwy', 'ad...",+16156141184,(615) 614-1184,55092.784846,NaN
1,8mEcWgU7q6lWchl-aJy_dw,the-city-forum-clarksville,The City Forum,https://s3-media3.fl.yelpcdn.com/bphoto/nLNur6...,False,https://www.yelp.com/biz/the-city-forum-clarks...,57,"[{'alias': 'mini_golf', 'title': 'Mini Golf'},...",4.0,"{'latitude': 36.511204, 'longitude': -87.2745814}",[],"{'address1': '2231-T Madison St', 'address2': ...",+19319195051,(931) 919-5051,33312.136770,NaN
2,3MuUUkRYnx56VAYyENvQDA,up-down-nashville-nashville-2,Up-Down Nashville,https://s3-media3.fl.yelpcdn.com/bphoto/JuPSXB...,False,https://www.yelp.com/biz/up-down-nashville-nas...,23,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 36.1759, 'longitude': -86.7546}","[pickup, delivery]","{'address1': '927 Woodland St', 'address2': No...",+16152263747,(615) 226-3747,31732.220217,$
4,rRsh-zolgfN96fz6UU_uzA,magic-wheels-fun-center-clarksville,Magic Wheels Fun Center,https://s3-media3.fl.yelpcdn.com/bphoto/YE170t...,False,https://www.yelp.com/biz/magic-wheels-fun-cent...,16,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.0,"{'latitude': 36.58135790157406, 'longitude': -...",[],"{'address1': '1671 Fort Campbell Blvd', 'addre...",+19319067300,(931) 906-7300,46433.313553,NaN
5,1NYbb6BzNjEyUwFKc9UfZw,game-terminal-nashville,Game Terminal,https://s3-media3.fl.yelpcdn.com/bphoto/PO3erT...,False,https://www.yelp.com/biz/game-terminal-nashvil...,63,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.131339492100224, 'longitude': ...",[],"{'address1': '201 Terminal Ct', 'address2': ''...",+16156102460,(615) 610-2460,34131.360677,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,ztQK4EfzwMbj3zzceiJxAA,tullahoma-bowling-lanes-tullahoma,Tullahoma Bowling Lanes,https://s3-media2.fl.yelpcdn.com/bphoto/iXGjte...,False,https://www.yelp.com/biz/tullahoma-bowling-lan...,7,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",2.5,"{'latitude': 35.3563, 'longitude': -86.19797}",[],"{'address1': '900 S Anderson St', 'address2': ...",+19314551947,(931) 455-1947,59492.132681,NaN
107,tHVupO1RIdd-DjZ-BbytWQ,ralphies-fun-center-glasgow,Ralphie's Fun Center,https://s3-media4.fl.yelpcdn.com/bphoto/yoZf_k...,False,https://www.yelp.com/biz/ralphies-fun-center-g...,9,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",4.5,"{'latitude': 37.0086209, 'longitude': -85.9189...",[],"{'address1': '702 Happy Valley Rd', 'address2'...",+12706294263,(270) 629-4263,79927.217044,NaN
108,rvimEgjXRq38ACp1wFXi_g,paradise-games-and-arcade-cookeville,Paradise Games and Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/dYpjpD...,False,https://www.yelp.com/biz/paradise-games-and-ar...,1,"[{'alias': 'videogamestores', 'title': 'Video ...",5.0,"{'latitude': 36.16233218340348, 'longitude': -...",[],"{'address1': '8 S Washington Ave', 'address2':...",+19312149263,(931) 214-9263,72450.315101,NaN
109,Jdx6joQwAE_byUINnYl1Jg,the-sk8-cookeville-2,The Sk8,https://s3-media1.fl.yelpcdn.com/bphoto/eH_2Mt...,False,https://www.yelp.com/biz/the-sk8-cookeville-2?...,4,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.5,"{'latitude': 36.1399541973511, 'longitude': -8...",[],"{'address1': '1810 Foreman Dr', 'address2': ''...",+19313722758,(931) 372-2758,69865.781632,NaN


In [51]:
arcades_df = arcades_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
arcades_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,UiURMNx6i-qeBWvwcIKPgQ,joyfull-arcade-spring-hill-2,Joyfull Arcade,https://s3-media2.fl.yelpcdn.com/bphoto/z5EZ5s...,False,https://www.yelp.com/biz/joyfull-arcade-spring...,1,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.77542748017264, 'longitude': -...",[],"{'address1': '2001 Campbell Station Pkwy', 'ad...",+16156141184,(615) 614-1184,55092.784846,NaN
1,8mEcWgU7q6lWchl-aJy_dw,the-city-forum-clarksville,The City Forum,https://s3-media3.fl.yelpcdn.com/bphoto/nLNur6...,False,https://www.yelp.com/biz/the-city-forum-clarks...,57,"[{'alias': 'mini_golf', 'title': 'Mini Golf'},...",4.0,"{'latitude': 36.511204, 'longitude': -87.2745814}",[],"{'address1': '2231-T Madison St', 'address2': ...",+19319195051,(931) 919-5051,33312.136770,NaN
2,3MuUUkRYnx56VAYyENvQDA,up-down-nashville-nashville-2,Up-Down Nashville,https://s3-media3.fl.yelpcdn.com/bphoto/JuPSXB...,False,https://www.yelp.com/biz/up-down-nashville-nas...,23,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 36.1759, 'longitude': -86.7546}","[pickup, delivery]","{'address1': '927 Woodland St', 'address2': No...",+16152263747,(615) 226-3747,31732.220217,$
3,rRsh-zolgfN96fz6UU_uzA,magic-wheels-fun-center-clarksville,Magic Wheels Fun Center,https://s3-media3.fl.yelpcdn.com/bphoto/YE170t...,False,https://www.yelp.com/biz/magic-wheels-fun-cent...,16,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.0,"{'latitude': 36.58135790157406, 'longitude': -...",[],"{'address1': '1671 Fort Campbell Blvd', 'addre...",+19319067300,(931) 906-7300,46433.313553,NaN
4,1NYbb6BzNjEyUwFKc9UfZw,game-terminal-nashville,Game Terminal,https://s3-media3.fl.yelpcdn.com/bphoto/PO3erT...,False,https://www.yelp.com/biz/game-terminal-nashvil...,63,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.131339492100224, 'longitude': ...",[],"{'address1': '201 Terminal Ct', 'address2': ''...",+16156102460,(615) 610-2460,34131.360677,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,ztQK4EfzwMbj3zzceiJxAA,tullahoma-bowling-lanes-tullahoma,Tullahoma Bowling Lanes,https://s3-media2.fl.yelpcdn.com/bphoto/iXGjte...,False,https://www.yelp.com/biz/tullahoma-bowling-lan...,7,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",2.5,"{'latitude': 35.3563, 'longitude': -86.19797}",[],"{'address1': '900 S Anderson St', 'address2': ...",+19314551947,(931) 455-1947,59492.132681,NaN
67,tHVupO1RIdd-DjZ-BbytWQ,ralphies-fun-center-glasgow,Ralphie's Fun Center,https://s3-media4.fl.yelpcdn.com/bphoto/yoZf_k...,False,https://www.yelp.com/biz/ralphies-fun-center-g...,9,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",4.5,"{'latitude': 37.0086209, 'longitude': -85.9189...",[],"{'address1': '702 Happy Valley Rd', 'address2'...",+12706294263,(270) 629-4263,79927.217044,NaN
68,rvimEgjXRq38ACp1wFXi_g,paradise-games-and-arcade-cookeville,Paradise Games and Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/dYpjpD...,False,https://www.yelp.com/biz/paradise-games-and-ar...,1,"[{'alias': 'videogamestores', 'title': 'Video ...",5.0,"{'latitude': 36.16233218340348, 'longitude': -...",[],"{'address1': '8 S Washington Ave', 'address2':...",+19312149263,(931) 214-9263,72450.315101,NaN
69,Jdx6joQwAE_byUINnYl1Jg,the-sk8-cookeville-2,The Sk8,https://s3-media1.fl.yelpcdn.com/bphoto/eH_2Mt...,False,https://www.yelp.com/biz/the-sk8-cookeville-2?...,4,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.5,"{'latitude': 36.1399541973511, 'longitude': -8...",[],"{'address1': '1810 Foreman Dr', 'address2': ''...",+19313722758,(931) 372-2758,69865.781632,NaN


In [52]:
arcades_df['category'] = 'other'
arcades_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,category
0,UiURMNx6i-qeBWvwcIKPgQ,joyfull-arcade-spring-hill-2,Joyfull Arcade,https://s3-media2.fl.yelpcdn.com/bphoto/z5EZ5s...,False,https://www.yelp.com/biz/joyfull-arcade-spring...,1,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 35.77542748017264, 'longitude': -...",[],"{'address1': '2001 Campbell Station Pkwy', 'ad...",+16156141184,(615) 614-1184,55092.784846,NaN,other
1,8mEcWgU7q6lWchl-aJy_dw,the-city-forum-clarksville,The City Forum,https://s3-media3.fl.yelpcdn.com/bphoto/nLNur6...,False,https://www.yelp.com/biz/the-city-forum-clarks...,57,"[{'alias': 'mini_golf', 'title': 'Mini Golf'},...",4.0,"{'latitude': 36.511204, 'longitude': -87.2745814}",[],"{'address1': '2231-T Madison St', 'address2': ...",+19319195051,(931) 919-5051,33312.136770,NaN,other
2,3MuUUkRYnx56VAYyENvQDA,up-down-nashville-nashville-2,Up-Down Nashville,https://s3-media3.fl.yelpcdn.com/bphoto/JuPSXB...,False,https://www.yelp.com/biz/up-down-nashville-nas...,23,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 36.1759, 'longitude': -86.7546}","[pickup, delivery]","{'address1': '927 Woodland St', 'address2': No...",+16152263747,(615) 226-3747,31732.220217,$,other
3,rRsh-zolgfN96fz6UU_uzA,magic-wheels-fun-center-clarksville,Magic Wheels Fun Center,https://s3-media3.fl.yelpcdn.com/bphoto/YE170t...,False,https://www.yelp.com/biz/magic-wheels-fun-cent...,16,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.0,"{'latitude': 36.58135790157406, 'longitude': -...",[],"{'address1': '1671 Fort Campbell Blvd', 'addre...",+19319067300,(931) 906-7300,46433.313553,NaN,other
4,1NYbb6BzNjEyUwFKc9UfZw,game-terminal-nashville,Game Terminal,https://s3-media3.fl.yelpcdn.com/bphoto/PO3erT...,False,https://www.yelp.com/biz/game-terminal-nashvil...,63,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",4.0,"{'latitude': 36.131339492100224, 'longitude': ...",[],"{'address1': '201 Terminal Ct', 'address2': ''...",+16156102460,(615) 610-2460,34131.360677,$$,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,ztQK4EfzwMbj3zzceiJxAA,tullahoma-bowling-lanes-tullahoma,Tullahoma Bowling Lanes,https://s3-media2.fl.yelpcdn.com/bphoto/iXGjte...,False,https://www.yelp.com/biz/tullahoma-bowling-lan...,7,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",2.5,"{'latitude': 35.3563, 'longitude': -86.19797}",[],"{'address1': '900 S Anderson St', 'address2': ...",+19314551947,(931) 455-1947,59492.132681,NaN,other
67,tHVupO1RIdd-DjZ-BbytWQ,ralphies-fun-center-glasgow,Ralphie's Fun Center,https://s3-media4.fl.yelpcdn.com/bphoto/yoZf_k...,False,https://www.yelp.com/biz/ralphies-fun-center-g...,9,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",4.5,"{'latitude': 37.0086209, 'longitude': -85.9189...",[],"{'address1': '702 Happy Valley Rd', 'address2'...",+12706294263,(270) 629-4263,79927.217044,NaN,other
68,rvimEgjXRq38ACp1wFXi_g,paradise-games-and-arcade-cookeville,Paradise Games and Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/dYpjpD...,False,https://www.yelp.com/biz/paradise-games-and-ar...,1,"[{'alias': 'videogamestores', 'title': 'Video ...",5.0,"{'latitude': 36.16233218340348, 'longitude': -...",[],"{'address1': '8 S Washington Ave', 'address2':...",+19312149263,(931) 214-9263,72450.315101,NaN,other
69,Jdx6joQwAE_byUINnYl1Jg,the-sk8-cookeville-2,The Sk8,https://s3-media1.fl.yelpcdn.com/bphoto/eH_2Mt...,False,https://www.yelp.com/biz/the-sk8-cookeville-2?...,4,"[{'alias': 'arcades', 'title': 'Arcades'}, {'a...",3.5,"{'latitude': 36.1399541973511, 'longitude': -8...",[],"{'address1': '1810 Foreman Dr', 'address2': ''...",+19313722758,(931) 372-2758,69865.781632,NaN,other


In [53]:
arcades_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_arcades.csv')

In [ ]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
family_friendly = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'family_friendly', location = location, search_limit = 20, offset = x)
        family_friendly.append(response)
        print(len(family_friendly))

In [ ]:
family_friendly_df = pd.DataFrame()

for family_friend in range(len(family_friendly)):
    df = pd.DataFrame(family_friendly[family_friend]['businesses'])
    family_friendly_df = family_friendly_df.append(df)
    print(family_friendly_df.shape)

In [ ]:
family_friendly_df = family_friendly_df.drop_duplicates(subset = 'name', keep = 'first')  
family_friendly_df = family_friendly_df.reset_index(drop=True)  
family_friendly_df

In [ ]:
family_friendly_df = family_friendly_df.drop([])
family_friendly_df

#not able to look through large df for this one. seeing restaurants, plumbing, mortage brokers. Will search just
#restaurants instead to give the option for families

In [ ]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
family_deals = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'family_deals', location = location, search_limit = 20, offset = x)
        family_deals.append(response)
        print(len(family_deals))

In [ ]:
family_deals_df = pd.DataFrame()

for family_deal in range(len(family_deals)):
    df = pd.DataFrame(family_deals[family_deal]['businesses'])
    family_deals_df = family_deals_df.append(df)
    print(family_deals_df.shape)

In [ ]:
family_deals_df = family_deals_df.drop_duplicates(subset = 'name', keep = 'first')  
family_deals_df = family_deals_df.reset_index(drop=True)  
family_deals_df

In [ ]:
family_deals_df = family_deals_df.drop([])
family_deals_df
#most are restaurants for family deals so will not use this and again use the restaurant search instead.

In [ ]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
children_free = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'children_free', location = location, search_limit = 20, offset = x)
        children_free.append(response)
        print(len(children_free))

In [ ]:
children_free_df = pd.DataFrame()

for children_fre in range(len(children_free)):
    df = pd.DataFrame(children_free[children_fre]['businesses'])
    children_free_df = children_free_df.append(df)
    print(children_free_df.shape)

In [ ]:
children_free_df = children_free_df.drop_duplicates(subset = 'name', keep = 'first')  
children_free_df = children_free_df.reset_index(drop=True)  
children_free_df

In [ ]:
children_free_df = children_free_df.drop([])
children_free_df

#not very helpful/does not seem to be children are free but a child care establishment. will not use but instead
#search for childcare for families

In [54]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
restaurants = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'restaurants', location = location, search_limit = 20, offset = x)
        restaurants.append(response)
        print(len(restaurants))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [55]:
restaurants_df = pd.DataFrame()

for restaurant in range(len(restaurants)):
    df = pd.DataFrame(restaurants[restaurant]['businesses'])
    restaurants_df = restaurants_df.append(df)
    print(restaurants_df.shape)

(20, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(29, 16)
(49, 16)
(69, 16)
(89, 16)
(109, 16)
(129, 16)
(149, 16)
(169, 16)
(189, 16)
(209, 16)
(229, 16)
(249, 16)
(269, 16)
(289, 16)
(309, 16)
(329, 16)
(349, 16)
(369, 16)
(389, 16)
(409, 16)
(429, 16)
(449, 16)
(469, 16)
(489, 16)
(509, 16)
(529, 16)
(549, 16)
(569, 16)
(589, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(601, 16)
(621, 16)
(641, 16)
(661, 16)
(681, 16)
(701, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(717, 16)
(737, 16)
(757, 16)
(777, 16

In [56]:
restaurants_df = restaurants_df.drop_duplicates(subset = 'name', keep = 'first')  
restaurants_df = restaurants_df.reset_index(drop=True)  
restaurants_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,CU7JcRlCu01SFdf6CCITXQ,riverview-restaurant-and-marina-ashland-city,Riverview Restaurant & Marina,https://s3-media1.fl.yelpcdn.com/bphoto/RcIVbT...,False,https://www.yelp.com/biz/riverview-restaurant-...,75,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 36.268783, 'longitude': -87.07734}",[],$$,"{'address1': '110 Old River Rd', 'address2': '...",+16157927358,(615) 792-7358,2899.348289
1,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],NaN,"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105
2,hjwblgY6usJNax7i2r1APg,marrowbone-creek-brewing-ashland-city-2,Marrowbone Creek Brewing,https://s3-media1.fl.yelpcdn.com/bphoto/xXBAoN...,False,https://www.yelp.com/biz/marrowbone-creek-brew...,14,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 36.27292284722772, 'longitude': -...",[],NaN,"{'address1': '240 S Main St', 'address2': None...",,,4109.917098
3,uOx0sKON3JcIB993Ra3w7w,vuocolos-italian-restaurant-and-wine-bar-ashla...,Vuocolo's Italian Restaurant and Wine Bar,https://s3-media3.fl.yelpcdn.com/bphoto/B4aMWx...,False,https://www.yelp.com/biz/vuocolos-italian-rest...,49,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.2713508605957, 'longitude': -8...",[],$$,"{'address1': '189 Monroe Pl', 'address2': 'Ste...",+16157927447,(615) 792-7447,5573.178739
4,aARsj1cawKPwea7--VqaRw,laredos-burrito-and-taco-shop-ashland-city,Laredo's Burrito and Taco Shop,https://s3-media2.fl.yelpcdn.com/bphoto/scblYH...,False,https://www.yelp.com/biz/laredos-burrito-and-t...,19,"[{'alias': 'mexican', 'title': 'Mexican'}]",5.0,"{'latitude': 36.27548, 'longitude': -87.065251}",[],NaN,"{'address1': '202 N Main St', 'address2': '', ...",+16152461155,(615) 246-1155,4158.811964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,dMh97wYOIhC5p6CB7X1r9A,davids-pizza-and-restaurant-lebanon,David's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-S_VlG...,False,https://www.yelp.com/biz/davids-pizza-and-rest...,25,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 36.21214, 'longitude': -86.29002}",[delivery],$$,"{'address1': '329 N Cumberland St', 'address2'...",+16154448232,(615) 444-8232,6519.968601
1420,vRdFTKtRdefl-r6ljm4BeA,cook-out-lebanon-2,Cook Out,https://s3-media3.fl.yelpcdn.com/bphoto/B0bXSK...,False,https://www.yelp.com/biz/cook-out-lebanon-2?ad...,10,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.5,"{'latitude': 36.189118, 'longitude': -86.295242}",[delivery],NaN,"{'address1': '107 Legends Dr', 'address2': '',...",,,3920.817416
1421,8w4ytXE0pIAy5jYsKNP43g,los-compadres-mexican-restaurant-lebanon,Los Compadres Mexican Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/qcGTnS...,False,https://www.yelp.com/biz/los-compadres-mexican...,42,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 36.208552, 'longitude': -86.30285}",[delivery],$,"{'address1': '512 W Main St', 'address2': '', ...",+16154434697,(615) 443-4697,5970.523707
1422,yg2GCzacSurz61pvK8ctEQ,dairy-queen-lebanon-9,Dairy Queen,https://s3-media2.fl.yelpcdn.com/bphoto/UriLR0...,False,https://www.yelp.com/biz/dairy-queen-lebanon-9...,4,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,"{'latitude': 36.179246881417434, 'longitude': ...",[delivery],$,"{'address1': '921 Murfreesboro Rd', 'address2'...",+16154538991,(615) 453-8991,2765.972152


In [57]:
restaurants_df = restaurants_df.drop([])
restaurants_df

#quick look, assuming all are restaurants. too large to go through.

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,CU7JcRlCu01SFdf6CCITXQ,riverview-restaurant-and-marina-ashland-city,Riverview Restaurant & Marina,https://s3-media1.fl.yelpcdn.com/bphoto/RcIVbT...,False,https://www.yelp.com/biz/riverview-restaurant-...,75,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 36.268783, 'longitude': -87.07734}",[],$$,"{'address1': '110 Old River Rd', 'address2': '...",+16157927358,(615) 792-7358,2899.348289
1,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],NaN,"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105
2,hjwblgY6usJNax7i2r1APg,marrowbone-creek-brewing-ashland-city-2,Marrowbone Creek Brewing,https://s3-media1.fl.yelpcdn.com/bphoto/xXBAoN...,False,https://www.yelp.com/biz/marrowbone-creek-brew...,14,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 36.27292284722772, 'longitude': -...",[],NaN,"{'address1': '240 S Main St', 'address2': None...",,,4109.917098
3,uOx0sKON3JcIB993Ra3w7w,vuocolos-italian-restaurant-and-wine-bar-ashla...,Vuocolo's Italian Restaurant and Wine Bar,https://s3-media3.fl.yelpcdn.com/bphoto/B4aMWx...,False,https://www.yelp.com/biz/vuocolos-italian-rest...,49,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.2713508605957, 'longitude': -8...",[],$$,"{'address1': '189 Monroe Pl', 'address2': 'Ste...",+16157927447,(615) 792-7447,5573.178739
4,aARsj1cawKPwea7--VqaRw,laredos-burrito-and-taco-shop-ashland-city,Laredo's Burrito and Taco Shop,https://s3-media2.fl.yelpcdn.com/bphoto/scblYH...,False,https://www.yelp.com/biz/laredos-burrito-and-t...,19,"[{'alias': 'mexican', 'title': 'Mexican'}]",5.0,"{'latitude': 36.27548, 'longitude': -87.065251}",[],NaN,"{'address1': '202 N Main St', 'address2': '', ...",+16152461155,(615) 246-1155,4158.811964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,dMh97wYOIhC5p6CB7X1r9A,davids-pizza-and-restaurant-lebanon,David's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-S_VlG...,False,https://www.yelp.com/biz/davids-pizza-and-rest...,25,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 36.21214, 'longitude': -86.29002}",[delivery],$$,"{'address1': '329 N Cumberland St', 'address2'...",+16154448232,(615) 444-8232,6519.968601
1420,vRdFTKtRdefl-r6ljm4BeA,cook-out-lebanon-2,Cook Out,https://s3-media3.fl.yelpcdn.com/bphoto/B0bXSK...,False,https://www.yelp.com/biz/cook-out-lebanon-2?ad...,10,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.5,"{'latitude': 36.189118, 'longitude': -86.295242}",[delivery],NaN,"{'address1': '107 Legends Dr', 'address2': '',...",,,3920.817416
1421,8w4ytXE0pIAy5jYsKNP43g,los-compadres-mexican-restaurant-lebanon,Los Compadres Mexican Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/qcGTnS...,False,https://www.yelp.com/biz/los-compadres-mexican...,42,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 36.208552, 'longitude': -86.30285}",[delivery],$,"{'address1': '512 W Main St', 'address2': '', ...",+16154434697,(615) 443-4697,5970.523707
1422,yg2GCzacSurz61pvK8ctEQ,dairy-queen-lebanon-9,Dairy Queen,https://s3-media2.fl.yelpcdn.com/bphoto/UriLR0...,False,https://www.yelp.com/biz/dairy-queen-lebanon-9...,4,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,"{'latitude': 36.179246881417434, 'longitude': ...",[delivery],$,"{'address1': '921 Murfreesboro Rd', 'address2'...",+16154538991,(615) 453-8991,2765.972152


In [58]:
restaurants_df = restaurants_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
restaurants_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,CU7JcRlCu01SFdf6CCITXQ,riverview-restaurant-and-marina-ashland-city,Riverview Restaurant & Marina,https://s3-media1.fl.yelpcdn.com/bphoto/RcIVbT...,False,https://www.yelp.com/biz/riverview-restaurant-...,75,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 36.268783, 'longitude': -87.07734}",[],$$,"{'address1': '110 Old River Rd', 'address2': '...",+16157927358,(615) 792-7358,2899.348289
1,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],NaN,"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105
2,hjwblgY6usJNax7i2r1APg,marrowbone-creek-brewing-ashland-city-2,Marrowbone Creek Brewing,https://s3-media1.fl.yelpcdn.com/bphoto/xXBAoN...,False,https://www.yelp.com/biz/marrowbone-creek-brew...,14,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 36.27292284722772, 'longitude': -...",[],NaN,"{'address1': '240 S Main St', 'address2': None...",,,4109.917098
3,uOx0sKON3JcIB993Ra3w7w,vuocolos-italian-restaurant-and-wine-bar-ashla...,Vuocolo's Italian Restaurant and Wine Bar,https://s3-media3.fl.yelpcdn.com/bphoto/B4aMWx...,False,https://www.yelp.com/biz/vuocolos-italian-rest...,49,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.2713508605957, 'longitude': -8...",[],$$,"{'address1': '189 Monroe Pl', 'address2': 'Ste...",+16157927447,(615) 792-7447,5573.178739
4,aARsj1cawKPwea7--VqaRw,laredos-burrito-and-taco-shop-ashland-city,Laredo's Burrito and Taco Shop,https://s3-media2.fl.yelpcdn.com/bphoto/scblYH...,False,https://www.yelp.com/biz/laredos-burrito-and-t...,19,"[{'alias': 'mexican', 'title': 'Mexican'}]",5.0,"{'latitude': 36.27548, 'longitude': -87.065251}",[],NaN,"{'address1': '202 N Main St', 'address2': '', ...",+16152461155,(615) 246-1155,4158.811964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,dMh97wYOIhC5p6CB7X1r9A,davids-pizza-and-restaurant-lebanon,David's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-S_VlG...,False,https://www.yelp.com/biz/davids-pizza-and-rest...,25,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 36.21214, 'longitude': -86.29002}",[delivery],$$,"{'address1': '329 N Cumberland St', 'address2'...",+16154448232,(615) 444-8232,6519.968601
1420,vRdFTKtRdefl-r6ljm4BeA,cook-out-lebanon-2,Cook Out,https://s3-media3.fl.yelpcdn.com/bphoto/B0bXSK...,False,https://www.yelp.com/biz/cook-out-lebanon-2?ad...,10,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.5,"{'latitude': 36.189118, 'longitude': -86.295242}",[delivery],NaN,"{'address1': '107 Legends Dr', 'address2': '',...",,,3920.817416
1421,8w4ytXE0pIAy5jYsKNP43g,los-compadres-mexican-restaurant-lebanon,Los Compadres Mexican Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/qcGTnS...,False,https://www.yelp.com/biz/los-compadres-mexican...,42,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 36.208552, 'longitude': -86.30285}",[delivery],$,"{'address1': '512 W Main St', 'address2': '', ...",+16154434697,(615) 443-4697,5970.523707
1422,yg2GCzacSurz61pvK8ctEQ,dairy-queen-lebanon-9,Dairy Queen,https://s3-media2.fl.yelpcdn.com/bphoto/UriLR0...,False,https://www.yelp.com/biz/dairy-queen-lebanon-9...,4,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,"{'latitude': 36.179246881417434, 'longitude': ...",[delivery],$,"{'address1': '921 Murfreesboro Rd', 'address2'...",+16154538991,(615) 453-8991,2765.972152


In [59]:
restaurants_df['category'] = 'restaurant'
restaurants_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,category
0,CU7JcRlCu01SFdf6CCITXQ,riverview-restaurant-and-marina-ashland-city,Riverview Restaurant & Marina,https://s3-media1.fl.yelpcdn.com/bphoto/RcIVbT...,False,https://www.yelp.com/biz/riverview-restaurant-...,75,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 36.268783, 'longitude': -87.07734}",[],$$,"{'address1': '110 Old River Rd', 'address2': '...",+16157927358,(615) 792-7358,2899.348289,restaurant
1,IGoGsMBEEz_axpBF0exk6A,o-brien-s-southern-diner-ashland-city,O’Brien’s Southern Diner,https://s3-media2.fl.yelpcdn.com/bphoto/4R9IgL...,False,https://www.yelp.com/biz/o-brien-s-southern-di...,30,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.28057, 'longitude': -87.14424}",[],NaN,"{'address1': '2905 Hwy 49 W', 'address2': '', ...",+16158457227,(615) 845-7227,5585.342105,restaurant
2,hjwblgY6usJNax7i2r1APg,marrowbone-creek-brewing-ashland-city-2,Marrowbone Creek Brewing,https://s3-media1.fl.yelpcdn.com/bphoto/xXBAoN...,False,https://www.yelp.com/biz/marrowbone-creek-brew...,14,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 36.27292284722772, 'longitude': -...",[],NaN,"{'address1': '240 S Main St', 'address2': None...",,,4109.917098,restaurant
3,uOx0sKON3JcIB993Ra3w7w,vuocolos-italian-restaurant-and-wine-bar-ashla...,Vuocolo's Italian Restaurant and Wine Bar,https://s3-media3.fl.yelpcdn.com/bphoto/B4aMWx...,False,https://www.yelp.com/biz/vuocolos-italian-rest...,49,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.2713508605957, 'longitude': -8...",[],$$,"{'address1': '189 Monroe Pl', 'address2': 'Ste...",+16157927447,(615) 792-7447,5573.178739,restaurant
4,aARsj1cawKPwea7--VqaRw,laredos-burrito-and-taco-shop-ashland-city,Laredo's Burrito and Taco Shop,https://s3-media2.fl.yelpcdn.com/bphoto/scblYH...,False,https://www.yelp.com/biz/laredos-burrito-and-t...,19,"[{'alias': 'mexican', 'title': 'Mexican'}]",5.0,"{'latitude': 36.27548, 'longitude': -87.065251}",[],NaN,"{'address1': '202 N Main St', 'address2': '', ...",+16152461155,(615) 246-1155,4158.811964,restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,dMh97wYOIhC5p6CB7X1r9A,davids-pizza-and-restaurant-lebanon,David's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-S_VlG...,False,https://www.yelp.com/biz/davids-pizza-and-rest...,25,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 36.21214, 'longitude': -86.29002}",[delivery],$$,"{'address1': '329 N Cumberland St', 'address2'...",+16154448232,(615) 444-8232,6519.968601,restaurant
1420,vRdFTKtRdefl-r6ljm4BeA,cook-out-lebanon-2,Cook Out,https://s3-media3.fl.yelpcdn.com/bphoto/B0bXSK...,False,https://www.yelp.com/biz/cook-out-lebanon-2?ad...,10,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.5,"{'latitude': 36.189118, 'longitude': -86.295242}",[delivery],NaN,"{'address1': '107 Legends Dr', 'address2': '',...",,,3920.817416,restaurant
1421,8w4ytXE0pIAy5jYsKNP43g,los-compadres-mexican-restaurant-lebanon,Los Compadres Mexican Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/qcGTnS...,False,https://www.yelp.com/biz/los-compadres-mexican...,42,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 36.208552, 'longitude': -86.30285}",[delivery],$,"{'address1': '512 W Main St', 'address2': '', ...",+16154434697,(615) 443-4697,5970.523707,restaurant
1422,yg2GCzacSurz61pvK8ctEQ,dairy-queen-lebanon-9,Dairy Queen,https://s3-media2.fl.yelpcdn.com/bphoto/UriLR0...,False,https://www.yelp.com/biz/dairy-queen-lebanon-9...,4,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,"{'latitude': 36.179246881417434, 'longitude': ...",[delivery],$,"{'address1': '921 Murfreesboro Rd', 'address2'...",+16154538991,(615) 453-8991,2765.972152,restaurant


In [60]:
restaurants_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_restaurants.csv')

In [61]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
childcare = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'childcare', location = location, search_limit = 20, offset = x)
        childcare.append(response)
        print(len(childcare))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [62]:
childcare_df = pd.DataFrame()

for childcar in range(len(childcare)):
    df = pd.DataFrame(childcare[childcar]['businesses'])
    childcare_df = childcare_df.append(df)
    print(childcare_df.shape)

(20, 15)
(40, 15)
(60, 15)
(80, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(83, 16)
(103, 16)
(123, 16)
(143, 16)
(163, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(166, 16)
(186, 16)
(206, 16)
(226, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(236, 16)
(256, 16)
(276, 16)
(296, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(300, 16)
(320, 16)
(340, 16)
(360,

In [63]:
childcare_df = childcare_df.drop_duplicates(subset = 'name', keep = 'first')  
childcare_df = childcare_df.reset_index(drop=True)  
childcare_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,KMszNiV6ume8M28nmKVCnA,cheekwood-child-development-center-nashville,Cheekwood Child Development Center,,False,https://www.yelp.com/biz/cheekwood-child-devel...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.08823, 'longitude': -86.88032}",[],"{'address1': '112 Cheekwood Ter', 'address2': ...",+16153539765,(615) 353-9765,26345.964336,NaN
1,QsCQy93r2AjUXIYLkRrkfQ,need-a-break-spring-hill-2,Need A Break,https://s3-media3.fl.yelpcdn.com/bphoto/0jxVf0...,False,https://www.yelp.com/biz/need-a-break-spring-h...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 35.77308, 'longitude': -86.91313}",[],"{'address1': '3011 Longford Dr', 'address2': '...",+16152411245,(615) 241-1245,55412.784910,NaN
2,uWIo6F9WpbW7lrNpAmlHIQ,the-children-s-playroom-drop-in-nashville,The Children’s Playroom Drop In,https://s3-media3.fl.yelpcdn.com/bphoto/OjzCDO...,False,https://www.yelp.com/biz/the-children-s-playro...,12,"[{'alias': 'childcare', 'title': 'Child Care &...",4.5,"{'latitude': 36.1037674, 'longitude': -86.8690...",[],"{'address1': '5133 Harding Pike', 'address2': ...",+16153715279,(615) 371-5279,25988.966133,NaN
3,OtVbTNqIaBUtBsULiqmrUQ,parents-timeout-clarksville,Parents Timeout,,False,https://www.yelp.com/biz/parents-timeout-clark...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.623315, 'longitude': -87.318965}",[],"{'address1': '3862 Trenton Rd', 'address2': 'S...",+19312669816,(931) 266-9816,46222.447672,NaN
4,_BLdpuC_zfYfDLuNSgwbPA,positive-image-learning-greenbrier-2,Positive Image Learning,https://s3-media2.fl.yelpcdn.com/bphoto/aLzBcB...,False,https://www.yelp.com/biz/positive-image-learni...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.398772, 'longitude': -86.76411}",[],"{'address1': '1805 Lake Rd', 'address2': '', '...",+16154208078,(615) 420-8078,34190.835983,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,sXnSzyNszZu2g4uLFcdF8A,stars-academy-rising-franklin,Stars Academy Rising,,False,https://www.yelp.com/biz/stars-academy-rising-...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 35.9200706, 'longitude': -86.8254...",[],"{'address1': '1215 Lakeview Dr', 'address2': N...",+16155676133,(615) 567-6133,43868.791975,NaN
79,6sT8JG5p2yByrHRhmX6IPA,all-my-sons-moving-and-storage-murfreesboro-3,All My Sons Moving & Storage,https://s3-media1.fl.yelpcdn.com/bphoto/bvJC6B...,False,https://www.yelp.com/biz/all-my-sons-moving-an...,47,"[{'alias': 'movers', 'title': 'Movers'}, {'ali...",1.5,"{'latitude': 35.84612, 'longitude': -86.4394}",[],"{'address1': '125 River Rock Blvd', 'address2'...",+16156176975,(615) 617-6975,74099.083117,NaN
80,mlpwOUYfJbI5h_DDCOeCiw,una-baptist-church-child-care-center-nashville,Una Baptist Church Child Care Center,,False,https://www.yelp.com/biz/una-baptist-church-ch...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 36.09578, 'longitude': -86.65741}",[],"{'address1': '1931 Murfreesboro Pike', 'addres...",+16153617507,(615) 361-7507,42961.890253,NaN
81,4eJLysMjtev2gctZNqnGeQ,aunty-ks-child-care-center-la-vergne-11,Aunty K's Child Care Center,https://s3-media2.fl.yelpcdn.com/bphoto/vo4Dfh...,False,https://www.yelp.com/biz/aunty-ks-child-care-c...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",2.5,"{'latitude': 36.0174499, 'longitude': -86.55725}",[],"{'address1': '203 Dick Buchanan St', 'address2...",+16152130499,(615) 213-0499,54822.461597,NaN


In [64]:
childcare_df[0:50]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,KMszNiV6ume8M28nmKVCnA,cheekwood-child-development-center-nashville,Cheekwood Child Development Center,,False,https://www.yelp.com/biz/cheekwood-child-devel...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.08823, 'longitude': -86.88032}",[],"{'address1': '112 Cheekwood Ter', 'address2': ...",+16153539765,(615) 353-9765,26345.964336,NaN
1,QsCQy93r2AjUXIYLkRrkfQ,need-a-break-spring-hill-2,Need A Break,https://s3-media3.fl.yelpcdn.com/bphoto/0jxVf0...,False,https://www.yelp.com/biz/need-a-break-spring-h...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 35.77308, 'longitude': -86.91313}",[],"{'address1': '3011 Longford Dr', 'address2': '...",+16152411245,(615) 241-1245,55412.784910,NaN
2,uWIo6F9WpbW7lrNpAmlHIQ,the-children-s-playroom-drop-in-nashville,The Children’s Playroom Drop In,https://s3-media3.fl.yelpcdn.com/bphoto/OjzCDO...,False,https://www.yelp.com/biz/the-children-s-playro...,12,"[{'alias': 'childcare', 'title': 'Child Care &...",4.5,"{'latitude': 36.1037674, 'longitude': -86.8690...",[],"{'address1': '5133 Harding Pike', 'address2': ...",+16153715279,(615) 371-5279,25988.966133,NaN
3,OtVbTNqIaBUtBsULiqmrUQ,parents-timeout-clarksville,Parents Timeout,,False,https://www.yelp.com/biz/parents-timeout-clark...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.623315, 'longitude': -87.318965}",[],"{'address1': '3862 Trenton Rd', 'address2': 'S...",+19312669816,(931) 266-9816,46222.447672,NaN
4,_BLdpuC_zfYfDLuNSgwbPA,positive-image-learning-greenbrier-2,Positive Image Learning,https://s3-media2.fl.yelpcdn.com/bphoto/aLzBcB...,False,https://www.yelp.com/biz/positive-image-learni...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.398772, 'longitude': -86.76411}",[],"{'address1': '1805 Lake Rd', 'address2': '', '...",+16154208078,(615) 420-8078,34190.835983,NaN
5,1nUkW9AA8asRTc4u4hLXww,little-heroes-academy-clarksville,Little Heroes Academy,https://s3-media1.fl.yelpcdn.com/bphoto/YVKqf7...,False,https://www.yelp.com/biz/little-heroes-academy...,3,"[{'alias': 'elementaryschools', 'title': 'Elem...",3.5,"{'latitude': 36.6256713867188, 'longitude': -8...",[],"{'address1': '1823 Tiny Town Rd', 'address2': ...",+19315030014,(931) 503-0014,46542.653993,NaN
6,Jhh-nbZF0wNnvaunKdxp6Q,the-childrens-playroom-drop-in-brentwood-10,The Children's Playroom Drop In,https://s3-media2.fl.yelpcdn.com/bphoto/wOH3aR...,False,https://www.yelp.com/biz/the-childrens-playroo...,10,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.02499626222008, 'longitude': -...",[],"{'address1': '330 Franklin Rd', 'address2': 'S...",+16153715279,(615) 371-5279,37061.753124,NaN
7,fx-F6yo-8deFH4Env3pulw,tlc-childcare-portland,TLC Childcare,,False,https://www.yelp.com/biz/tlc-childcare-portlan...,2,"[{'alias': 'preschools', 'title': 'Preschools'...",5.0,"{'latitude': 36.5848236083984, 'longitude': -8...",[],"{'address1': '755 College St', 'address2': '',...",+16157455171,(615) 745-5171,61751.623161,NaN
8,pIRsnvsQxTvX-cf9THTRBA,little-professional-learning-center-nashville,Little Professional Learning Center,https://s3-media1.fl.yelpcdn.com/bphoto/w1YOie...,False,https://www.yelp.com/biz/little-professional-l...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.1703606, 'longitude': -86.6740...",[],"{'address1': '228 Shady Grove Dr', 'address2':...",+16159020777,(615) 902-0777,38925.204170,NaN
9,zg9bTWxxkcgprPLuBgMndQ,the-gardner-school-of-brentwood-brentwood-2,The Gardner School of Brentwood,https://s3-media1.fl.yelpcdn.com/bphoto/toXg3e...,False,https://www.yelp.com/biz/the-gardner-school-of...,6,"[{'alias': 'preschools', 'title': 'Preschools'...",4.5,"{'latitude': 35.97628190303782, 'longitude': -...",[],"{'address1': '1591 Mallory Ln', 'address2': No...",+16156130472,(615) 613-0

In [65]:
childcare_df = childcare_df.drop([30,53,56,64,67,75,77,79])
childcare_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,KMszNiV6ume8M28nmKVCnA,cheekwood-child-development-center-nashville,Cheekwood Child Development Center,,False,https://www.yelp.com/biz/cheekwood-child-devel...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.08823, 'longitude': -86.88032}",[],"{'address1': '112 Cheekwood Ter', 'address2': ...",+16153539765,(615) 353-9765,26345.964336,NaN
1,QsCQy93r2AjUXIYLkRrkfQ,need-a-break-spring-hill-2,Need A Break,https://s3-media3.fl.yelpcdn.com/bphoto/0jxVf0...,False,https://www.yelp.com/biz/need-a-break-spring-h...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 35.77308, 'longitude': -86.91313}",[],"{'address1': '3011 Longford Dr', 'address2': '...",+16152411245,(615) 241-1245,55412.784910,NaN
2,uWIo6F9WpbW7lrNpAmlHIQ,the-children-s-playroom-drop-in-nashville,The Children’s Playroom Drop In,https://s3-media3.fl.yelpcdn.com/bphoto/OjzCDO...,False,https://www.yelp.com/biz/the-children-s-playro...,12,"[{'alias': 'childcare', 'title': 'Child Care &...",4.5,"{'latitude': 36.1037674, 'longitude': -86.8690...",[],"{'address1': '5133 Harding Pike', 'address2': ...",+16153715279,(615) 371-5279,25988.966133,NaN
3,OtVbTNqIaBUtBsULiqmrUQ,parents-timeout-clarksville,Parents Timeout,,False,https://www.yelp.com/biz/parents-timeout-clark...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.623315, 'longitude': -87.318965}",[],"{'address1': '3862 Trenton Rd', 'address2': 'S...",+19312669816,(931) 266-9816,46222.447672,NaN
4,_BLdpuC_zfYfDLuNSgwbPA,positive-image-learning-greenbrier-2,Positive Image Learning,https://s3-media2.fl.yelpcdn.com/bphoto/aLzBcB...,False,https://www.yelp.com/biz/positive-image-learni...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.398772, 'longitude': -86.76411}",[],"{'address1': '1805 Lake Rd', 'address2': '', '...",+16154208078,(615) 420-8078,34190.835983,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Nxge7mf3DSSllB3DZzEGtQ,donelson-hermitage-family-ymca-nashville,Donelson-Hermitage Family YMCA,https://s3-media3.fl.yelpcdn.com/bphoto/xWr0zP...,False,https://www.yelp.com/biz/donelson-hermitage-fa...,30,"[{'alias': 'gyms', 'title': 'Gyms'}, {'alias':...",3.5,"{'latitude': 36.171861, 'longitude': -86.648633}",[],"{'address1': '3001 Lebanon Rd', 'address2': ''...",+16158892632,(615) 889-2632,41095.883456,NaN
78,sXnSzyNszZu2g4uLFcdF8A,stars-academy-rising-franklin,Stars Academy Rising,,False,https://www.yelp.com/biz/stars-academy-rising-...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 35.9200706, 'longitude': -86.8254...",[],"{'address1': '1215 Lakeview Dr', 'address2': N...",+16155676133,(615) 567-6133,43868.791975,NaN
80,mlpwOUYfJbI5h_DDCOeCiw,una-baptist-church-child-care-center-nashville,Una Baptist Church Child Care Center,,False,https://www.yelp.com/biz/una-baptist-church-ch...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 36.09578, 'longitude': -86.65741}",[],"{'address1': '1931 Murfreesboro Pike', 'addres...",+16153617507,(615) 361-7507,42961.890253,NaN
81,4eJLysMjtev2gctZNqnGeQ,aunty-ks-child-care-center-la-vergne-11,Aunty K's Child Care Center,https://s3-media2.fl.yelpcdn.com/bphoto/vo4Dfh...,False,https://www.yelp.com/biz/aunty-ks-child-care-c...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",2.5,"{'latitude': 36.0174499, 'longitude': -86.55725}",[],"{'address1': '203 Dick Buchanan St', 'address2...",+16152130499,(615) 213-0499,54822.461597,NaN


In [66]:
childcare_df = childcare_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
childcare_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,KMszNiV6ume8M28nmKVCnA,cheekwood-child-development-center-nashville,Cheekwood Child Development Center,,False,https://www.yelp.com/biz/cheekwood-child-devel...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.08823, 'longitude': -86.88032}",[],"{'address1': '112 Cheekwood Ter', 'address2': ...",+16153539765,(615) 353-9765,26345.964336,NaN
1,QsCQy93r2AjUXIYLkRrkfQ,need-a-break-spring-hill-2,Need A Break,https://s3-media3.fl.yelpcdn.com/bphoto/0jxVf0...,False,https://www.yelp.com/biz/need-a-break-spring-h...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 35.77308, 'longitude': -86.91313}",[],"{'address1': '3011 Longford Dr', 'address2': '...",+16152411245,(615) 241-1245,55412.784910,NaN
2,uWIo6F9WpbW7lrNpAmlHIQ,the-children-s-playroom-drop-in-nashville,The Children’s Playroom Drop In,https://s3-media3.fl.yelpcdn.com/bphoto/OjzCDO...,False,https://www.yelp.com/biz/the-children-s-playro...,12,"[{'alias': 'childcare', 'title': 'Child Care &...",4.5,"{'latitude': 36.1037674, 'longitude': -86.8690...",[],"{'address1': '5133 Harding Pike', 'address2': ...",+16153715279,(615) 371-5279,25988.966133,NaN
3,OtVbTNqIaBUtBsULiqmrUQ,parents-timeout-clarksville,Parents Timeout,,False,https://www.yelp.com/biz/parents-timeout-clark...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.623315, 'longitude': -87.318965}",[],"{'address1': '3862 Trenton Rd', 'address2': 'S...",+19312669816,(931) 266-9816,46222.447672,NaN
4,_BLdpuC_zfYfDLuNSgwbPA,positive-image-learning-greenbrier-2,Positive Image Learning,https://s3-media2.fl.yelpcdn.com/bphoto/aLzBcB...,False,https://www.yelp.com/biz/positive-image-learni...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.398772, 'longitude': -86.76411}",[],"{'address1': '1805 Lake Rd', 'address2': '', '...",+16154208078,(615) 420-8078,34190.835983,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Nxge7mf3DSSllB3DZzEGtQ,donelson-hermitage-family-ymca-nashville,Donelson-Hermitage Family YMCA,https://s3-media3.fl.yelpcdn.com/bphoto/xWr0zP...,False,https://www.yelp.com/biz/donelson-hermitage-fa...,30,"[{'alias': 'gyms', 'title': 'Gyms'}, {'alias':...",3.5,"{'latitude': 36.171861, 'longitude': -86.648633}",[],"{'address1': '3001 Lebanon Rd', 'address2': ''...",+16158892632,(615) 889-2632,41095.883456,NaN
71,sXnSzyNszZu2g4uLFcdF8A,stars-academy-rising-franklin,Stars Academy Rising,,False,https://www.yelp.com/biz/stars-academy-rising-...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 35.9200706, 'longitude': -86.8254...",[],"{'address1': '1215 Lakeview Dr', 'address2': N...",+16155676133,(615) 567-6133,43868.791975,NaN
72,mlpwOUYfJbI5h_DDCOeCiw,una-baptist-church-child-care-center-nashville,Una Baptist Church Child Care Center,,False,https://www.yelp.com/biz/una-baptist-church-ch...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 36.09578, 'longitude': -86.65741}",[],"{'address1': '1931 Murfreesboro Pike', 'addres...",+16153617507,(615) 361-7507,42961.890253,NaN
73,4eJLysMjtev2gctZNqnGeQ,aunty-ks-child-care-center-la-vergne-11,Aunty K's Child Care Center,https://s3-media2.fl.yelpcdn.com/bphoto/vo4Dfh...,False,https://www.yelp.com/biz/aunty-ks-child-care-c...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",2.5,"{'latitude': 36.0174499, 'longitude': -86.55725}",[],"{'address1': '203 Dick Buchanan St', 'address2...",+16152130499,(615) 213-0499,54822.461597,NaN


In [67]:
childcare_df['category'] = 'childcare'
childcare_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,category
0,KMszNiV6ume8M28nmKVCnA,cheekwood-child-development-center-nashville,Cheekwood Child Development Center,,False,https://www.yelp.com/biz/cheekwood-child-devel...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.08823, 'longitude': -86.88032}",[],"{'address1': '112 Cheekwood Ter', 'address2': ...",+16153539765,(615) 353-9765,26345.964336,NaN,childcare
1,QsCQy93r2AjUXIYLkRrkfQ,need-a-break-spring-hill-2,Need A Break,https://s3-media3.fl.yelpcdn.com/bphoto/0jxVf0...,False,https://www.yelp.com/biz/need-a-break-spring-h...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 35.77308, 'longitude': -86.91313}",[],"{'address1': '3011 Longford Dr', 'address2': '...",+16152411245,(615) 241-1245,55412.784910,NaN,childcare
2,uWIo6F9WpbW7lrNpAmlHIQ,the-children-s-playroom-drop-in-nashville,The Children’s Playroom Drop In,https://s3-media3.fl.yelpcdn.com/bphoto/OjzCDO...,False,https://www.yelp.com/biz/the-children-s-playro...,12,"[{'alias': 'childcare', 'title': 'Child Care &...",4.5,"{'latitude': 36.1037674, 'longitude': -86.8690...",[],"{'address1': '5133 Harding Pike', 'address2': ...",+16153715279,(615) 371-5279,25988.966133,NaN,childcare
3,OtVbTNqIaBUtBsULiqmrUQ,parents-timeout-clarksville,Parents Timeout,,False,https://www.yelp.com/biz/parents-timeout-clark...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",5.0,"{'latitude': 36.623315, 'longitude': -87.318965}",[],"{'address1': '3862 Trenton Rd', 'address2': 'S...",+19312669816,(931) 266-9816,46222.447672,NaN,childcare
4,_BLdpuC_zfYfDLuNSgwbPA,positive-image-learning-greenbrier-2,Positive Image Learning,https://s3-media2.fl.yelpcdn.com/bphoto/aLzBcB...,False,https://www.yelp.com/biz/positive-image-learni...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",4.0,"{'latitude': 36.398772, 'longitude': -86.76411}",[],"{'address1': '1805 Lake Rd', 'address2': '', '...",+16154208078,(615) 420-8078,34190.835983,NaN,childcare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Nxge7mf3DSSllB3DZzEGtQ,donelson-hermitage-family-ymca-nashville,Donelson-Hermitage Family YMCA,https://s3-media3.fl.yelpcdn.com/bphoto/xWr0zP...,False,https://www.yelp.com/biz/donelson-hermitage-fa...,30,"[{'alias': 'gyms', 'title': 'Gyms'}, {'alias':...",3.5,"{'latitude': 36.171861, 'longitude': -86.648633}",[],"{'address1': '3001 Lebanon Rd', 'address2': ''...",+16158892632,(615) 889-2632,41095.883456,NaN,childcare
71,sXnSzyNszZu2g4uLFcdF8A,stars-academy-rising-franklin,Stars Academy Rising,,False,https://www.yelp.com/biz/stars-academy-rising-...,1,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 35.9200706, 'longitude': -86.8254...",[],"{'address1': '1215 Lakeview Dr', 'address2': N...",+16155676133,(615) 567-6133,43868.791975,NaN,childcare
72,mlpwOUYfJbI5h_DDCOeCiw,una-baptist-church-child-care-center-nashville,Una Baptist Church Child Care Center,,False,https://www.yelp.com/biz/una-baptist-church-ch...,2,"[{'alias': 'childcare', 'title': 'Child Care &...",1.0,"{'latitude': 36.09578, 'longitude': -86.65741}",[],"{'address1': '1931 Murfreesboro Pike', 'addres...",+16153617507,(615) 361-7507,42961.890253,NaN,childcare
73,4eJLysMjtev2gctZNqnGeQ,aunty-ks-child-care-center-la-vergne-11,Aunty K's Child Care Center,https://s3-media2.fl.yelpcdn.com/bphoto/vo4Dfh...,False,https://www.yelp.com/biz/aunty-ks-child-care-c...,3,"[{'alias': 'childcare', 'title': 'Child Care &...",2.5,"{'latitude': 36.0174499, 'longitude': -86.55725}",[],"{'address1': '203 Dick Buchanan St', 'address2...",+16152130499,(615) 213-0499,54822.461597,NaN,childcare


In [68]:
childcare_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_childcare.csv')

In [69]:
locations = ['Cheatham County, TN', 'Davidson County, TN', 'Dickson County, TN', 'Maury County, TN', 'Montgomery County, TN', 'Robertson County, TN', 'Rutherford County, TN', 'Sumner County, TN', 'Williamson County, TN', 'Wilson County, TN']
group = np.arange(0,500,20)
movie_theaters = []

for location in locations:
    for x in group:
        response = yelp_api.search_query(term = 'movie theater', location = location, search_limit = 20, offset = x)
        movie_theaters.append(response)
        print(len(movie_theaters))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [70]:
movie_theaters_df = pd.DataFrame()

for movie_theater in range(len(movie_theaters)):
    df = pd.DataFrame(movie_theaters[movie_theater]['businesses'])
    movie_theaters_df = movie_theaters_df.append(df)
    print(movie_theaters_df.shape)

(20, 16)
(40, 16)
(60, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(64, 16)
(84, 16)
(104, 16)
(124, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(130, 16)
(150, 16)
(170, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(185, 16)
(205, 16)
(225, 16)
(245, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(246, 16)
(266, 16)
(286, 16)
(303, 

In [71]:
movie_theaters_df = movie_theaters_df.drop_duplicates(subset = 'name', keep = 'first')  
movie_theaters_df = movie_theaters_df.reset_index(drop=True)  
movie_theaters_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,4jISUD7S_gjIG0XWD4DO0Q,amc-bellevue-12-nashville,AMC Bellevue 12,https://s3-media3.fl.yelpcdn.com/bphoto/mNSB1j...,False,https://www.yelp.com/biz/amc-bellevue-12-nashv...,68,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",4.0,"{'latitude': 36.081019, 'longitude': -86.947374}",[],"{'address1': '8125 Sawyer Brown Road', 'addres...",+16156780424,(615) 678-0424,22909.039498,NaN
1,GG4_Idv210PI992wl4Dp_A,regal-green-hills-nashville-4,Regal Green Hills,https://s3-media1.fl.yelpcdn.com/bphoto/fIBuV4...,False,https://www.yelp.com/biz/regal-green-hills-nas...,149,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.0,"{'latitude': 36.108659, 'longitude': -86.81711}",[],"{'address1': '3815 Greenhills Village Drive', ...",+18444627342,(844) 462-7342,29504.569532,NaN
2,zINmA47xaKHQWg19v3KPlw,regal-clarksville-and-rpx-clarksville-4,Regal Clarksville & RPX,https://s3-media4.fl.yelpcdn.com/bphoto/ydTvDD...,False,https://www.yelp.com/biz/regal-clarksville-and...,82,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.6235084533691, 'longitude': -8...",[],"{'address1': '1810 Tiny Town Road', 'address2'...",+18444627342,(844) 462-7342,46292.753769,NaN
3,KqF1W-GxTAnOZtcMNS5Wiw,belcourt-theatre-nashville,Belcourt Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/7RUZE0...,False,https://www.yelp.com/biz/belcourt-theatre-nash...,211,"[{'alias': 'movietheaters', 'title': 'Cinema'}...",4.5,"{'latitude': 36.136915, 'longitude': -86.801515}",[],"{'address1': '2102 Belcourt Ave', 'address2': ...",+16158463150,(615) 846-3150,29248.400842,$
4,Ug61Y4JGo2ddyIv6OzZT9A,bellevue-cinema-twelve-nashville,Bellevue Cinema Twelve,https://s3-media3.fl.yelpcdn.com/bphoto/Z3ugi5...,False,https://www.yelp.com/biz/bellevue-cinema-twelv...,3,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.0809173583984, 'longitude': -8...",[],"{'address1': '7741 Highway 70 S', 'address2': ...",+16156463111,(615) 646-3111,22279.605528,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Sk5OlkXiIg3jZlm6suKJ9Q,montana-drive-in-tullahoma,Montana Drive-In,,False,https://www.yelp.com/biz/montana-drive-in-tull...,9,"[{'alias': 'driveintheater', 'title': 'Drive-I...",2.5,"{'latitude': 35.311775, 'longitude': -86.166158}",[],"{'address1': '10251 Tullahoma Hwy', 'address2'...",+19316492270,(931) 649-2270,65115.859105,NaN
83,xcVebymOToBC8NHoktH04A,marquee-cinemas-highland-8-glasgow,Marquee Cinemas - Highland 8,https://s3-media2.fl.yelpcdn.com/bphoto/0nki2S...,False,https://www.yelp.com/biz/marquee-cinemas-highl...,14,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",2.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781605,(270) 678-1605,80478.514011,NaN
84,YsH5dW41aYSsTN0Unn_Ieg,plaza-theatre-glasgow,Plaza Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/8Yq-ZP...,False,https://www.yelp.com/biz/plaza-theatre-glasgow...,3,"[{'alias': 'theater', 'title': 'Performing Art...",4.5,"{'latitude': 36.9960632324219, 'longitude': -8...",[],"{'address1': '115 E Main St', 'address2': None...",+12703612101,(270) 361-2101,79293.243599,NaN
85,_cEg8Ec4K17uoTVladx5ZA,highland-7-cinema-glasgow,Highland 7 Cinema,,False,https://www.yelp.com/biz/highland-7-cinema-gla...,2,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",1.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781613,(270) 678-1613,80478.514011,NaN


In [72]:
movie_theaters_df[50:88]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
50,jSYu2apG1uYq3eNzSA-OHw,johnny-cash-museum-nashville-4,Johnny Cash Museum,https://s3-media2.fl.yelpcdn.com/bphoto/qkanvY...,False,https://www.yelp.com/biz/johnny-cash-museum-na...,432,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.16084, 'longitude': -86.77586}",[],"{'address1': '119 3rd Ave S', 'address2': '', ...",+16152561777,(615) 256-1777,30388.543084,NaN
51,xlop4czMJSsNcayLhfQ33Q,gaylord-opryland-resort-arcade-nashville,Gaylord Opryland Resort Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/kIGxHy...,False,https://www.yelp.com/biz/gaylord-opryland-reso...,1,"[{'alias': 'arcades', 'title': 'Arcades'}]",5.0,"{'latitude': 36.21261944535672, 'longitude': -...",[],"{'address1': '2800 Opryland Dr', 'address2': '...",+16158891000,(615) 889-1000,36410.203805,NaN
52,DjCCD8xRKAJmM_r-kB1jOA,war-memorial-auditorium-nashville,War Memorial Auditorium,https://s3-media1.fl.yelpcdn.com/bphoto/0MCI9d...,False,https://www.yelp.com/biz/war-memorial-auditori...,37,"[{'alias': 'musicvenues', 'title': 'Music Venu...",4.0,"{'latitude': 36.16410593120085, 'longitude': -...",[],"{'address1': '301 6th Ave N', 'address2': '', ...",+16157824000,(615) 782-4000,29631.109816,$$
53,czJM3pm_cAbz8Ohr7vGtJQ,dave-and-busters-nashville,Dave & Buster's,https://s3-media1.fl.yelpcdn.com/bphoto/jtsu7S...,False,https://www.yelp.com/biz/dave-and-busters-nash...,292,"[{'alias': 'tradamerican', 'title': 'American ...",2.5,"{'latitude': 36.2059619697926, 'longitude': -8...","[pickup, delivery]","{'address1': '540 Opry Mills Dr', 'address2': ...",+16159703800,(615) 970-3800,36410.578029,$$
54,rryEC-u2GnR0Z2_l_bUXtA,andrew-jacksons-hermitage-nashville-2,Andrew Jackson's Hermitage,https://s3-media4.fl.yelpcdn.com/bphoto/Mnsz3I...,False,https://www.yelp.com/biz/andrew-jacksons-hermi...,301,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.5,"{'latitude': 36.2150124, 'longitude': -86.6129...",[],"{'address1': '4580 Rachel's Ln', 'address2': '...",+16158892941,(615) 889-2941,43558.610105,NaN
55,yJkQP1Q5BP6TF6RShRSGpQ,nashville-childrens-theatre-nashville,Nashville Children's Theatre,https://s3-media3.fl.yelpcdn.com/bphoto/d07PqW...,False,https://www.yelp.com/biz/nashville-childrens-t...,10,"[{'alias': 'theater', 'title': 'Performing Art...",4.5,"{'latitude': 36.154592, 'longitude': -86.768063}",[],"{'address1': '25 Middleton St', 'address2': ''...",+16152549103,(615) 254-9103,31277.422268,NaN
56,PL7WdNXxZ1tJz7BHzCS66A,madame-tussauds-nashville-nashville,Madame Tussauds Nashville,https://s3-media4.fl.yelpcdn.com/bphoto/tLBTKR...,False,https://www.yelp.com/biz/madame-tussauds-nashv...,50,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.204849, 'longitude': -86.694068}",[],"{'address1': '515 Opry Mills Dr', 'address2': ...",+18668413263,(866) 841-3263,36447.627194,NaN
57,efGqAYPCCxPvHPX3dCM8bQ,tennessee-performing-arts-center-nashville-2,Tennessee Performing Arts Center,https://s3-media2.fl.yelpcdn.com/bphoto/rVxgMR...,False,https://www.yelp.com/biz/tennessee-performing-...,101,"[{'alias': 'theater', 'title': 'Performing Art...",4.0,"{'latitude': 36.16437, 'longitude': -86.782409}",[],"{'address1': '505 Deaderick St', 'address2': '...",+16157824040,(615) 782-4040,29706.531809,$$
58,O_l8AIlR2RJvNPq4CDMbtQ,bourbon-street-blues-and-boogie-bar-nashville,Bourbon Street Blues & Boogie Bar,https://s3-media3.fl.yelpcdn.com/bphoto/UIkIAm...,False,https://www.yelp.com/biz/bourbon-street-blues-...,438,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.0,"{'latitude': 36.1644533, 'longitude': -86.778836}","[pickup, delivery]","{'address1': '220 Printer's Aly', 'address2': ...",+16152425837,(615) 242-5837,30008.284472,$$
59,jPj-6M9NXbw9dsujjkPpFg,nashville-opera-nashville,Nashville Opera,https://s3-media2.fl.yelpcdn.com/bphoto/5YGZXs...,False,https://www.yelp.com/biz/nashville-opera-nashv...,3,"[{'alias': 'th

In [73]:
movie_theaters_df = movie_theaters_df.drop([30,31,35,39,40,42,43,44,45,46,47,48,49,50,51,52,53,55,57,59,60,61,62,63,74,79])
movie_theaters_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,4jISUD7S_gjIG0XWD4DO0Q,amc-bellevue-12-nashville,AMC Bellevue 12,https://s3-media3.fl.yelpcdn.com/bphoto/mNSB1j...,False,https://www.yelp.com/biz/amc-bellevue-12-nashv...,68,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",4.0,"{'latitude': 36.081019, 'longitude': -86.947374}",[],"{'address1': '8125 Sawyer Brown Road', 'addres...",+16156780424,(615) 678-0424,22909.039498,NaN
1,GG4_Idv210PI992wl4Dp_A,regal-green-hills-nashville-4,Regal Green Hills,https://s3-media1.fl.yelpcdn.com/bphoto/fIBuV4...,False,https://www.yelp.com/biz/regal-green-hills-nas...,149,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.0,"{'latitude': 36.108659, 'longitude': -86.81711}",[],"{'address1': '3815 Greenhills Village Drive', ...",+18444627342,(844) 462-7342,29504.569532,NaN
2,zINmA47xaKHQWg19v3KPlw,regal-clarksville-and-rpx-clarksville-4,Regal Clarksville & RPX,https://s3-media4.fl.yelpcdn.com/bphoto/ydTvDD...,False,https://www.yelp.com/biz/regal-clarksville-and...,82,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.6235084533691, 'longitude': -8...",[],"{'address1': '1810 Tiny Town Road', 'address2'...",+18444627342,(844) 462-7342,46292.753769,NaN
3,KqF1W-GxTAnOZtcMNS5Wiw,belcourt-theatre-nashville,Belcourt Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/7RUZE0...,False,https://www.yelp.com/biz/belcourt-theatre-nash...,211,"[{'alias': 'movietheaters', 'title': 'Cinema'}...",4.5,"{'latitude': 36.136915, 'longitude': -86.801515}",[],"{'address1': '2102 Belcourt Ave', 'address2': ...",+16158463150,(615) 846-3150,29248.400842,$
4,Ug61Y4JGo2ddyIv6OzZT9A,bellevue-cinema-twelve-nashville,Bellevue Cinema Twelve,https://s3-media3.fl.yelpcdn.com/bphoto/Z3ugi5...,False,https://www.yelp.com/biz/bellevue-cinema-twelv...,3,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.0809173583984, 'longitude': -8...",[],"{'address1': '7741 Highway 70 S', 'address2': ...",+16156463111,(615) 646-3111,22279.605528,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Sk5OlkXiIg3jZlm6suKJ9Q,montana-drive-in-tullahoma,Montana Drive-In,,False,https://www.yelp.com/biz/montana-drive-in-tull...,9,"[{'alias': 'driveintheater', 'title': 'Drive-I...",2.5,"{'latitude': 35.311775, 'longitude': -86.166158}",[],"{'address1': '10251 Tullahoma Hwy', 'address2'...",+19316492270,(931) 649-2270,65115.859105,NaN
83,xcVebymOToBC8NHoktH04A,marquee-cinemas-highland-8-glasgow,Marquee Cinemas - Highland 8,https://s3-media2.fl.yelpcdn.com/bphoto/0nki2S...,False,https://www.yelp.com/biz/marquee-cinemas-highl...,14,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",2.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781605,(270) 678-1605,80478.514011,NaN
84,YsH5dW41aYSsTN0Unn_Ieg,plaza-theatre-glasgow,Plaza Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/8Yq-ZP...,False,https://www.yelp.com/biz/plaza-theatre-glasgow...,3,"[{'alias': 'theater', 'title': 'Performing Art...",4.5,"{'latitude': 36.9960632324219, 'longitude': -8...",[],"{'address1': '115 E Main St', 'address2': None...",+12703612101,(270) 361-2101,79293.243599,NaN
85,_cEg8Ec4K17uoTVladx5ZA,highland-7-cinema-glasgow,Highland 7 Cinema,,False,https://www.yelp.com/biz/highland-7-cinema-gla...,2,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",1.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781613,(270) 678-1613,80478.514011,NaN


In [74]:
movie_theaters_df = movie_theaters_df.reset_index(drop=True)    #reset index and dropped previous index so it did not become a column
movie_theaters_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,4jISUD7S_gjIG0XWD4DO0Q,amc-bellevue-12-nashville,AMC Bellevue 12,https://s3-media3.fl.yelpcdn.com/bphoto/mNSB1j...,False,https://www.yelp.com/biz/amc-bellevue-12-nashv...,68,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",4.0,"{'latitude': 36.081019, 'longitude': -86.947374}",[],"{'address1': '8125 Sawyer Brown Road', 'addres...",+16156780424,(615) 678-0424,22909.039498,NaN
1,GG4_Idv210PI992wl4Dp_A,regal-green-hills-nashville-4,Regal Green Hills,https://s3-media1.fl.yelpcdn.com/bphoto/fIBuV4...,False,https://www.yelp.com/biz/regal-green-hills-nas...,149,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.0,"{'latitude': 36.108659, 'longitude': -86.81711}",[],"{'address1': '3815 Greenhills Village Drive', ...",+18444627342,(844) 462-7342,29504.569532,NaN
2,zINmA47xaKHQWg19v3KPlw,regal-clarksville-and-rpx-clarksville-4,Regal Clarksville & RPX,https://s3-media4.fl.yelpcdn.com/bphoto/ydTvDD...,False,https://www.yelp.com/biz/regal-clarksville-and...,82,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.6235084533691, 'longitude': -8...",[],"{'address1': '1810 Tiny Town Road', 'address2'...",+18444627342,(844) 462-7342,46292.753769,NaN
3,KqF1W-GxTAnOZtcMNS5Wiw,belcourt-theatre-nashville,Belcourt Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/7RUZE0...,False,https://www.yelp.com/biz/belcourt-theatre-nash...,211,"[{'alias': 'movietheaters', 'title': 'Cinema'}...",4.5,"{'latitude': 36.136915, 'longitude': -86.801515}",[],"{'address1': '2102 Belcourt Ave', 'address2': ...",+16158463150,(615) 846-3150,29248.400842,$
4,Ug61Y4JGo2ddyIv6OzZT9A,bellevue-cinema-twelve-nashville,Bellevue Cinema Twelve,https://s3-media3.fl.yelpcdn.com/bphoto/Z3ugi5...,False,https://www.yelp.com/biz/bellevue-cinema-twelv...,3,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.0809173583984, 'longitude': -8...",[],"{'address1': '7741 Highway 70 S', 'address2': ...",+16156463111,(615) 646-3111,22279.605528,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Sk5OlkXiIg3jZlm6suKJ9Q,montana-drive-in-tullahoma,Montana Drive-In,,False,https://www.yelp.com/biz/montana-drive-in-tull...,9,"[{'alias': 'driveintheater', 'title': 'Drive-I...",2.5,"{'latitude': 35.311775, 'longitude': -86.166158}",[],"{'address1': '10251 Tullahoma Hwy', 'address2'...",+19316492270,(931) 649-2270,65115.859105,NaN
57,xcVebymOToBC8NHoktH04A,marquee-cinemas-highland-8-glasgow,Marquee Cinemas - Highland 8,https://s3-media2.fl.yelpcdn.com/bphoto/0nki2S...,False,https://www.yelp.com/biz/marquee-cinemas-highl...,14,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",2.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781605,(270) 678-1605,80478.514011,NaN
58,YsH5dW41aYSsTN0Unn_Ieg,plaza-theatre-glasgow,Plaza Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/8Yq-ZP...,False,https://www.yelp.com/biz/plaza-theatre-glasgow...,3,"[{'alias': 'theater', 'title': 'Performing Art...",4.5,"{'latitude': 36.9960632324219, 'longitude': -8...",[],"{'address1': '115 E Main St', 'address2': None...",+12703612101,(270) 361-2101,79293.243599,NaN
59,_cEg8Ec4K17uoTVladx5ZA,highland-7-cinema-glasgow,Highland 7 Cinema,,False,https://www.yelp.com/biz/highland-7-cinema-gla...,2,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",1.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781613,(270) 678-1613,80478.514011,NaN


In [75]:
movie_theaters_df['category'] = 'theater'
movie_theaters_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,category
0,4jISUD7S_gjIG0XWD4DO0Q,amc-bellevue-12-nashville,AMC Bellevue 12,https://s3-media3.fl.yelpcdn.com/bphoto/mNSB1j...,False,https://www.yelp.com/biz/amc-bellevue-12-nashv...,68,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",4.0,"{'latitude': 36.081019, 'longitude': -86.947374}",[],"{'address1': '8125 Sawyer Brown Road', 'addres...",+16156780424,(615) 678-0424,22909.039498,NaN,theater
1,GG4_Idv210PI992wl4Dp_A,regal-green-hills-nashville-4,Regal Green Hills,https://s3-media1.fl.yelpcdn.com/bphoto/fIBuV4...,False,https://www.yelp.com/biz/regal-green-hills-nas...,149,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.0,"{'latitude': 36.108659, 'longitude': -86.81711}",[],"{'address1': '3815 Greenhills Village Drive', ...",+18444627342,(844) 462-7342,29504.569532,NaN,theater
2,zINmA47xaKHQWg19v3KPlw,regal-clarksville-and-rpx-clarksville-4,Regal Clarksville & RPX,https://s3-media4.fl.yelpcdn.com/bphoto/ydTvDD...,False,https://www.yelp.com/biz/regal-clarksville-and...,82,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.6235084533691, 'longitude': -8...",[],"{'address1': '1810 Tiny Town Road', 'address2'...",+18444627342,(844) 462-7342,46292.753769,NaN,theater
3,KqF1W-GxTAnOZtcMNS5Wiw,belcourt-theatre-nashville,Belcourt Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/7RUZE0...,False,https://www.yelp.com/biz/belcourt-theatre-nash...,211,"[{'alias': 'movietheaters', 'title': 'Cinema'}...",4.5,"{'latitude': 36.136915, 'longitude': -86.801515}",[],"{'address1': '2102 Belcourt Ave', 'address2': ...",+16158463150,(615) 846-3150,29248.400842,$,theater
4,Ug61Y4JGo2ddyIv6OzZT9A,bellevue-cinema-twelve-nashville,Bellevue Cinema Twelve,https://s3-media3.fl.yelpcdn.com/bphoto/Z3ugi5...,False,https://www.yelp.com/biz/bellevue-cinema-twelv...,3,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",3.5,"{'latitude': 36.0809173583984, 'longitude': -8...",[],"{'address1': '7741 Highway 70 S', 'address2': ...",+16156463111,(615) 646-3111,22279.605528,NaN,theater
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Sk5OlkXiIg3jZlm6suKJ9Q,montana-drive-in-tullahoma,Montana Drive-In,,False,https://www.yelp.com/biz/montana-drive-in-tull...,9,"[{'alias': 'driveintheater', 'title': 'Drive-I...",2.5,"{'latitude': 35.311775, 'longitude': -86.166158}",[],"{'address1': '10251 Tullahoma Hwy', 'address2'...",+19316492270,(931) 649-2270,65115.859105,NaN,theater
57,xcVebymOToBC8NHoktH04A,marquee-cinemas-highland-8-glasgow,Marquee Cinemas - Highland 8,https://s3-media2.fl.yelpcdn.com/bphoto/0nki2S...,False,https://www.yelp.com/biz/marquee-cinemas-highl...,14,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",2.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781605,(270) 678-1605,80478.514011,NaN,theater
58,YsH5dW41aYSsTN0Unn_Ieg,plaza-theatre-glasgow,Plaza Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/8Yq-ZP...,False,https://www.yelp.com/biz/plaza-theatre-glasgow...,3,"[{'alias': 'theater', 'title': 'Performing Art...",4.5,"{'latitude': 36.9960632324219, 'longitude': -8...",[],"{'address1': '115 E Main St', 'address2': None...",+12703612101,(270) 361-2101,79293.243599,NaN,theater
59,_cEg8Ec4K17uoTVladx5ZA,highland-7-cinema-glasgow,Highland 7 Cinema,,False,https://www.yelp.com/biz/highland-7-cinema-gla...,2,"[{'alias': 'movietheaters', 'title': 'Cinema'}]",1.0,"{'latitude': 37.02444, 'longitude': -85.93432}",[],"{'address1': '2400 Happy Valley Rd', 'address2...",+12706781613,(270) 678-1613,80478.514011,NaN,theater


In [76]:
movie_theaters_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_movie_theaters.csv')

In [ ]:
dflist = [parks_df, museums_df, libraries_df, arcades_df, restaurants_df, childcare_df,movie_theaters_df]

places_df = pd.concat(dflist)
places_df

In [ ]:
places_df = places_df.drop_duplicates(subset = 'name', keep = 'first')  
places_df = places_df.reset_index(drop=True)  
places_df

In [ ]:
places_df.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/yelp_places.csv')

In [ ]:
pov_data.to_csv('C:/Users/kahar/Documents/NSS/Capstone/low_cost_family_activities/Data/pov_data.csv')